
<h2 align="center"><strong> Redes Neuronales Convolucionales (CNN) utilizando Python</strong></h2>


Por: Joan Esteban López Narváez, Semillero ARES - Grupo GADyM, Universidad del Valle.
Contacto: joan.narvaez@correounivalle.edu.co

Versión 1.0

<p align="center">
    <img src="red.png" alt="Diagrama de la arquitectura de una red neuronal convolucional" style="max-width:40%; height:auto;">
</p>

Imagen tomada de : https://www.youtube.com/watch?v=dEXPMQXoiLc&list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3&index=7


En las siguientes secciones se presentarán, de forma gradual, los conceptos clave para comprender las redes neuronales convolucionales (CNN): neuronas, optimización, retropropagación (backpropagation) y otros temas relacionados. Cada concepto se construirá paso a paso de manera manual para afianzar la intuición, y posteriormente se mostrará cómo aprovechar librerías como TensorFlow y NumPy para implementar y optimizar arquitecturas CNN.

Si se busca más profundidad, se pueden afianzar los conceptos usando fuentes como lo son el Deep Learning Book, de Yoshua Bengio et Al., de acceso libre en el siguiente link: https://www.deeplearningbook.org/

Además, un buen recurso visual se encuentra en el siguiente curso corto (en el que se basó el contenido principal del libro computacional): https://www.youtube.com/watch?v=omz_NdFgWyU&list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3&index=6


## Tabla de Contenido

- [Introduccion](#introduccion)
- [Codificacion de una capa](#codificacion-de-una-capa)
- [Producto Escalar](#producto-escalar)
- [Lotes, capas y objetos](#lotes-capas-y-objetos)
- [Funciones de activacion](#funciones-de-activacion)
- [Activacion softmax](#activacion-softmax)
- [Calculo de perdidas](#calculo-de-perdidas)
- [Optimizacion](#optimizacion)
- [Implementacion en Tensorflow](#Implementacion-en-Tensorflow)




## Introduccion

Las **redes neuronales convolucionales (CNN, *Convolutional Neural Networks*)** constituyen una de las arquitecturas más influyentes dentro del campo del *deep learning* y el procesamiento de información visual.  

Su éxito se debe a la capacidad de **extraer automáticamente características jerárquicas** de los datos, lo que les permite reconocer patrones complejos en imágenes, señales o secuencias sin requerir una etapa explícita de ingeniería de características.

El principio fundamental de las CNN radica en el uso de **operaciones de convolución**, las cuales actúan como filtros capaces de detectar estructuras locales tales como bordes, texturas o regiones específicas de interés. Al combinar múltiples capas convolucionales, seguidas de operaciones de activación y reducción de dimensionalidad, la red puede construir representaciones cada vez más abstractas y robustas del conjunto de entrada.

Históricamente, las CNN surgieron como una evolución de los modelos neuronales clásicos inspirados en la organización del **córtex visual biológico**, y ganaron popularidad tras el éxito de **LeNet-5** (LeCun et al., 1998) en el reconocimiento de dígitos manuscritos. A partir de entonces, arquitecturas más profundas como **AlexNet**, **VGG**, **ResNet** e **Inception** impulsaron de manera decisiva el progreso en tareas de visión por computador, extendiendo su aplicación a otros dominios como el análisis de señales biomédicas, la visión robótica o el procesamiento de audio.

En este cuaderno se abordarán, de forma progresiva, los **conceptos fundamentales necesarios para comprender y construir una CNN desde cero**. Se explorarán las nociones de **neuronas, funciones de activación, optimización y retropropagación (backpropagation)**, entre otros elementos esenciales.  
Cada tema será desarrollado **paso a paso y de manera manual** para reforzar la intuición matemática y computacional, y posteriormente se mostrará cómo **implementar estos mismos principios utilizando librerías especializadas como NumPy y TensorFlow**, facilitando el diseño y entrenamiento de arquitecturas convolucionales modernas.

### Fundamentos de una red neuronal artificial  

Una **red neuronal artificial (ANN)** es un modelo computacional inspirado en el funcionamiento del cerebro humano. Su estructura básica está formada por **neuronas artificiales**, unidades matemáticas que procesan información. Cada neurona recibe un conjunto de **entradas** $x_1, x_2, \ldots, x_n$, las **multiplica por pesos** $w_1, w_2, \ldots, w_n$ (es decir, realiza productos $w_i x_i$), suma los resultados y aplica una **función de activación** $f(\cdot)$, produciendo una salida

$$
y = f\!\left(\sum_i w_i x_i + b\right)
$$

donde $b$ es un **sesgo**.

Este proceso puede parecer matemáticamente complejo, pero no es nada más que un simple ciclo for en el que, en una variable, se guarda la suma de la multiplicación de cada peso por cada una de las entradas dadas, tal y como se evidencia a continuación:

In [1]:
pesos = [0.1,2,3.5]
entradas = [1,2,3]
sesgo = 5.7
salida = 0

for i in range(2):
    salida = salida + pesos[i] * entradas[i]

salida = salida + sesgo

print(f'la salida de la neurona es {salida}')

la salida de la neurona es 9.8


Las neuronas se organizan en **capas**, que no son nada más ni nada menos que una **agrupación de neuronas**, cada una con sus propios pesos asignados:  
- La **capa de entrada** recibe los datos sin procesar (por ejemplo, los píxeles de una imagen).  
- Las **capas ocultas** realizan transformaciones lineales (multiplicación por pesos y suma) seguidas de no linealidades (activaciones). Es en estas capas donde se extraen y combinan características: los **pesos** son exactamente los parámetros que determinan cómo se valoran las entradas y **son los que se ajustan** durante el entrenamiento para que la red aprenda.  
- La **capa de salida** genera la predicción final; suele tener **n neuronas** (una por clase), y la neurona con la mayor activación indica la **decisión final** del modelo.

El proceso de entrenamiento busca ajustar esos pesos (y sesgos) minimizando una función de pérdida entre la salida predicha y la verdad de referencia, de modo que la red mejore su capacidad para reconocer patrones. A continuación, se presenta una lúdica interactiva que permite variar los pesos mediante un slider para ver el cambio en la salida:

In [27]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Valores originales
pesos_originales = [0.1, 2, 3.5]
entradas = [1, 2, 3]
sesgo = 5.7

def calcular_y_visualizar(mult_peso0, mult_peso1, mult_peso2):
    """Calcula y visualiza la salida de la neurona"""
    pesos_modificados = [
        pesos_originales[0] * mult_peso0,
        pesos_originales[1] * mult_peso1,
        pesos_originales[2] * mult_peso2
    ]
    
    # Calcular contribuciones
    contribuciones = [pesos_modificados[i] * entradas[i] for i in range(len(pesos_modificados))]
    salida = sum(contribuciones) + sesgo
    
    # Crear visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfica 1: Contribuciones por peso
    colores = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    bars1 = ax1.bar(range(len(contribuciones)), contribuciones, color=colores, alpha=0.7)
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
    ax1.set_xlabel('Índice del Peso', fontsize=12)
    ax1.set_ylabel('Contribución a la Salida', fontsize=12)
    ax1.set_title('Contribución de cada Peso × Entrada', fontsize=14, fontweight='bold', pad=20)
    ax1.set_xticks(range(len(contribuciones)))
    ax1.grid(axis='y', alpha=0.3)
    
    # Calcular límites y-axis con margen para las etiquetas
    max_val = max(contribuciones) if contribuciones else 0
    min_val = min(contribuciones) if contribuciones else 0
    rango = max_val - min_val
    margen = rango * 0.15 if rango > 0 else 1
    ax1.set_ylim(min_val - margen, max_val + margen)
    
    # Añadir valores en las barras
    for i, v in enumerate(contribuciones):
        offset = 0.3 if abs(v) > 0.5 else 0.15
        ax1.text(i, v + offset if v > 0 else v - offset, f'{v:.2f}', 
                ha='center', va='bottom' if v > 0 else 'top', fontweight='bold', fontsize=10)
    
    # Gráfica 2: Composición de la salida
    componentes = contribuciones + [sesgo]
    labels = [f'Peso {i}×Entrada {i}' for i in range(len(contribuciones))] + ['Sesgo']
    colores_comp = colores + ['#95E1D3']
    
    bars2 = ax2.barh(labels, componentes, color=colores_comp, alpha=0.7)
    ax2.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
    ax2.set_xlabel('Valor', fontsize=12)
    ax2.set_title(f'Composición de la Salida = {salida:.3f}', fontsize=14, fontweight='bold', pad=20)
    ax2.grid(axis='x', alpha=0.3)
    
    # Calcular límites x-axis con margen para las etiquetas
    max_comp = max(componentes) if componentes else 0
    min_comp = min(componentes) if componentes else 0
    rango_comp = max_comp - min_comp
    margen_comp = rango_comp * 0.2 if rango_comp > 0 else 1
    ax2.set_xlim(min_comp - margen_comp, max_comp + margen_comp)
    
    # Añadir valores
    for i, v in enumerate(componentes):
        offset = 0.5 if abs(v) > 1 else 0.3
        ax2.text(v + offset if v > 0 else v - offset, i, f'{v:.2f}', 
                va='center', ha='left' if v > 0 else 'right', fontweight='bold', fontsize=10)
    
    plt.tight_layout(pad=2.0)
    plt.show()
    
    # Mostrar información numérica
    print("=" * 60)
    print(f" SALIDA FINAL DE LA NEURONA: {salida:.3f}")
    print("=" * 60)
    print(f"Pesos modificados: [{pesos_modificados[0]:.3f}, {pesos_modificados[1]:.3f}, {pesos_modificados[2]:.3f}]")
    print(f"Multiplicadores:   [{mult_peso0:.2f}, {mult_peso1:.2f}, {mult_peso2:.2f}]")

# Crear sliders
slider_peso0 = widgets.FloatSlider(value=1.0, min=-1.0, max=1.0, step=0.05, 
                                   description='Mult. Peso 0:', continuous_update=True)
slider_peso1 = widgets.FloatSlider(value=1.0, min=-1.0, max=1.0, step=0.05, 
                                   description='Mult. Peso 1:', continuous_update=True)
slider_peso2 = widgets.FloatSlider(value=1.0, min=-1.0, max=1.0, step=0.05, 
                                   description='Mult. Peso 2:', continuous_update=True)

# Crear interfaz interactiva
widgets.interact(calcular_y_visualizar,
                mult_peso0=slider_peso0,
                mult_peso1=slider_peso1,
                mult_peso2=slider_peso2)

interactive(children=(FloatSlider(value=1.0, description='Mult. Peso 0:', max=1.0, min=-1.0, step=0.05), Float…

<function __main__.calcular_y_visualizar(mult_peso0, mult_peso1, mult_peso2)>

## Codificacion de una capa

Tras lo expuesto anteriormente, es natural que surjan algunas preguntas: ¿cómo se organizan realmente esas agrupaciones de capas de neuronas?, y más aún, ¿cómo pueden implementarse en Python, considerando que son las responsables del proceso de inferencia una vez reciben una entrada específica?
A lo largo de este capítulo se abordarán estas cuestiones mediante la implementación paso a paso de una red neuronal de una sola capa, con el objetivo de comprender en detalle su funcionamiento interno.

## Red neuronal de una capa oculta  

Hasta ahora se ha visto cómo una única neurona combina varias entradas ponderadas por sus pesos, suma un sesgo y produce una salida. Sin embargo, en la práctica, las redes neuronales constan de **múltiples neuronas organizadas en capas**, donde la **salida de una capa sirve como entrada para la siguiente**.  

Para comprender mejor esta estructura, construiremos una pequeña red con:
- **4 entradas**,  
- **una capa oculta con 3 neuronas**, y  
- **una capa de salida con una única neurona**.

El flujo de información será el siguiente:

$$
\text{Entradas} \rightarrow \text{Capa oculta (3 neuronas)} \rightarrow \text{Capa de salida (1 neurona)}
$$

Cada neurona en la capa oculta calcula una salida independiente según:

$$
y_j = \sum_{i=1}^{4} (x_i \cdot w_{ij}) + b_j
$$

donde:
- $x_i$ son las entradas,  
- $w_{ij}$ son los pesos que conectan la entrada $i$ con la neurona $j$,  
- $b_j$ es el sesgo de la neurona $j$.

Posteriormente, la **neurona de salida** tomará esas tres salidas de la capa oculta como sus entradas, aplicando el mismo principio.

---

### Implementación paso a paso en Python

A continuación se muestra el código para calcular la salida final de la red (sin usar librerías externas):

In [26]:
# Entradas de la red (4 valores)

entradas = [1.0, 2.0, 3.0, 2.5]

# Pesos de la capa oculta (3 neuronas × 4 entradas)

pesos_capa_oculta = [

    [0.2, 0.8, -0.5, 1.0],   # Neurona 1

    [0.5, -0.91, 0.26, -0.5],# Neurona 2

    [-0.26, -0.27, 0.17, 0.87]# Neurona 3

]

# Sesgos de la capa oculta (uno por neurona)

sesgos_capa_oculta = [2.0, 3.0, 0.5]

# Cálculo de la salida de la capa oculta

salidas_ocultas = []

for i in range(3):  # tres neuronas

    salida_neurona = 0

    for j in range(4):  # cuatro entradas

        salida_neurona += entradas[j] * pesos_capa_oculta[i][j]

    salida_neurona += sesgos_capa_oculta[i]

    salidas_ocultas.append(salida_neurona)

print("Salidas de la capa oculta:", salidas_ocultas)

# -------------------------------

# Capa de salida (una sola neurona)

# -------------------------------

pesos_salida = [0.3, -0.2, 0.8]  # tres pesos, uno por salida oculta

sesgo_salida = 1.5

# Cálculo de la salida final

salida_final = 0

for i in range(3):

    salida_final += salidas_ocultas[i] * pesos_salida[i]

salida_final += sesgo_salida

print(f"\nSalida final de la red: {salida_final:.4f}")

Salidas de la capa oculta: [4.8, 1.21, 2.385]

Salida final de la red: 4.6060


## Producto Escalar

Una manera de optimizar las operaciones realizadas anteriormente consiste en representar los pesos y bias de cada una de las neuronas como vectores (que son listas de elementos), matrices (que son listas de múltiples elementos organizados en filas y columnas) o tensores (que pueden entenderse como una extensión de las matrices a más de dos dimensiones; por ejemplo, un tensor de tres dimensiones puede imaginarse como un conjunto de matrices apiladas una sobre otra, y un tensor de cuatro dimensiones como un conjunto de esos bloques a lo largo de una nueva dimensión).

De esta manera, al utilizar el producto punto y, de manera más general, el producto entre matrices, es posible expresar los cálculos neuronales de forma compacta y eficiente, evitando el uso de múltiples bucles y reduciendo considerablemente el tiempo de ejecución.

Para ello, se utilizará la librería NumPy de Python, la cual ofrece funciones altamente optimizadas para realizar operaciones vectoriales y matriciales. Gracias a esta herramienta, el proceso de cálculo de la salida de una neurona (o incluso de una capa completa) puede representarse en apenas una línea de código, mejorando tanto la legibilidad como el rendimiento del programa.

En esencia, lo que intentamos calcular para encontrar la salida de una neurona (o de una capa completa) puede expresarse de manera general como:  

$$
\text{salida} = \text{entradas} \times \text{pesos} + \text{bias}
$$

donde el término $\text{entradas} \times \text{pesos}$ representa el **producto punto** entre ambos vectores.  
Esta operación resume el proceso de multiplicar cada entrada por su peso correspondiente y luego sumar todos los resultados.  
A este valor se le añade el **bias**, que permite ajustar la salida desplazando el resultado final.  
Gracias a **NumPy**, este cálculo puede implementarse en una sola línea, como se muestra a continuación para el ejemplo inicial de una única neurona:


In [4]:
import numpy as np

# Definición de entradas, pesos y bias
entradas = np.array([1, 2, 3])
pesos = np.array([0.1, 2, 3.5])
bias = 5.7

# Cálculo de la salida de la neurona
salida = np.dot(entradas, pesos) + bias

print(f'La salida de la neurona es {salida}')


La salida de la neurona es 20.3


Otro ejemplo un poco más visual del proceso que se lleva a cabo es el siguiente. Abajo se pueden ajustar los pesos de cada capa y visualizar gráficamente que es lo que está ocurriendo en cada paso.

In [25]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

# Configuración de la red
entradas = np.array([1.0, 2.0, 3.0])

# Pesos y sesgos de la capa oculta (3 neuronas)
pesos_oculta_originales = np.array([
    [0.2, 0.8, -0.5],   # Pesos para neurona oculta 0
    [0.5, -0.91, 0.26], # Pesos para neurona oculta 1
    [-0.26, -0.27, 0.17] # Pesos para neurona oculta 2
])
sesgos_oculta = np.array([2.0, 3.0, 0.5])

# Pesos y sesgos de la capa de salida (2 neuronas)
pesos_salida_originales = np.array([
    [0.1, 0.14, -0.5],  # Pesos para neurona salida 0
    [-0.3, 0.2, 0.9]    # Pesos para neurona salida 1
])
sesgos_salida = np.array([-1.0, 1.0])

def calcular_y_visualizar(mult_oculta, mult_salida):
    """Calcula y visualiza el flujo de información en la red neuronal"""
    
    # Aplicar multiplicadores a los pesos
    pesos_oculta = pesos_oculta_originales * mult_oculta
    pesos_salida = pesos_salida_originales * mult_salida
    
    # CAPA OCULTA: Calcular salidas de las 3 neuronas ocultas
    salidas_oculta = np.array([
        np.dot(entradas, pesos_oculta[0]) + sesgos_oculta[0],
        np.dot(entradas, pesos_oculta[1]) + sesgos_oculta[1],
        np.dot(entradas, pesos_oculta[2]) + sesgos_oculta[2]
    ])
    
    # CAPA DE SALIDA: Calcular salidas de las 2 neuronas de salida
    salidas_finales = np.array([
        np.dot(salidas_oculta, pesos_salida[0]) + sesgos_salida[0],
        np.dot(salidas_oculta, pesos_salida[1]) + sesgos_salida[1]
    ])
    
    # Crear visualización
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.4, wspace=0.3)
    
    # --- GRÁFICA 1: Arquitectura de la red ---
    ax1 = fig.add_subplot(gs[0, :])
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 10)
    ax1.axis('off')
    ax1.set_title('Arquitectura de la Red Neuronal', fontsize=16, fontweight='bold', pad=20)
    
    # Dibujar conexiones primero (para que queden detrás)
    # Conexiones entrada → oculta
    for i in range(3):
        for j in range(3):
            ax1.plot([1.9, 4.6], [7 - i*2.5, 7 - j*2.5], color='#FFA500', alpha=0.5, linewidth=1.5)
    # Conexiones oculta → salida
    for i in range(3):
        for j in range(2):
            ax1.plot([5.4, 8.1], [7 - i*2.5, 6 - j*3], color='#9370DB', alpha=0.5, linewidth=1.5)
    
    # Dibujar capas encima de las conexiones
    # Capa de entrada
    for i in range(3):
        circle = plt.Circle((1.5, 7 - i*2.5), 0.4, color='#FF6B6B', alpha=0.8, zorder=3)
        ax1.add_patch(circle)
        ax1.text(1.5, 7 - i*2.5, f'x{i}', ha='center', va='center', fontweight='bold', fontsize=10, zorder=4)
        ax1.text(0.3, 7 - i*2.5, f'{entradas[i]:.1f}', ha='center', va='center', fontsize=9)
    
    # Capa oculta
    for i in range(3):
        circle = plt.Circle((5, 7 - i*2.5), 0.4, color='#4ECDC4', alpha=0.8, zorder=3)
        ax1.add_patch(circle)
        ax1.text(5, 7 - i*2.5, f'h{i}', ha='center', va='center', fontweight='bold', fontsize=10, zorder=4)
        ax1.text(6.2, 7 - i*2.5, f'{salidas_oculta[i]:.2f}', ha='center', va='center', 
                fontsize=9, bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))
    
    # Capa de salida
    for i in range(2):
        circle = plt.Circle((8.5, 6 - i*3), 0.4, color='#45B7D1', alpha=0.8, zorder=3)
        ax1.add_patch(circle)
        ax1.text(8.5, 6 - i*3, f'y{i}', ha='center', va='center', fontweight='bold', fontsize=10, zorder=4)
        ax1.text(9.7, 6 - i*3, f'{salidas_finales[i]:.2f}', ha='center', va='center', 
                fontsize=9, bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.5))
    
    # Etiquetas de capas
    ax1.text(1.5, 9, 'Entrada', ha='center', fontsize=12, fontweight='bold')
    ax1.text(5, 9, 'Capa Oculta', ha='center', fontsize=12, fontweight='bold')
    ax1.text(8.5, 9, 'Salida', ha='center', fontsize=12, fontweight='bold')
    
    # --- GRÁFICA 2: Salidas de la capa oculta ---
    ax2 = fig.add_subplot(gs[1, 0])
    colores_oculta = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    bars_oculta = ax2.bar(range(3), salidas_oculta, color=colores_oculta, alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
    ax2.set_xlabel('Neurona Oculta', fontsize=11)
    ax2.set_ylabel('Valor de Salida', fontsize=11)
    ax2.set_title('Salidas de la Capa Oculta', fontsize=13, fontweight='bold', pad=15)
    ax2.set_xticks(range(3))
    ax2.set_xticklabels(['h0', 'h1', 'h2'])
    ax2.grid(axis='y', alpha=0.3)
    
    # Calcular límites con margen
    max_oculta = np.max(salidas_oculta) if len(salidas_oculta) > 0 else 1
    min_oculta = np.min(salidas_oculta) if len(salidas_oculta) > 0 else 0
    rango_oculta = max_oculta - min_oculta
    margen_oculta = rango_oculta * 0.2 if rango_oculta > 0 else 1
    ax2.set_ylim(min_oculta - margen_oculta, max_oculta + margen_oculta)
    
    # Añadir valores
    for i, v in enumerate(salidas_oculta):
        offset = rango_oculta * 0.05 if rango_oculta > 0 else 0.1
        ax2.text(i, v + offset if v > 0 else v - offset, f'{v:.2f}', 
                ha='center', va='bottom' if v > 0 else 'top', fontweight='bold', fontsize=9)
    
    # --- GRÁFICA 3: Salidas finales ---
    ax3 = fig.add_subplot(gs[1, 1])
    colores_salida = ['#95E1D3', '#F38181']
    bars_salida = ax3.bar(range(2), salidas_finales, color=colores_salida, alpha=0.7)
    ax3.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
    ax3.set_xlabel('Neurona de Salida', fontsize=11)
    ax3.set_ylabel('Valor de Salida', fontsize=11)
    ax3.set_title('Salidas Finales de la Red', fontsize=13, fontweight='bold', pad=15)
    ax3.set_xticks(range(2))
    ax3.set_xticklabels(['y0', 'y1'])
    ax3.grid(axis='y', alpha=0.3)
    
    # Calcular límites con margen
    max_salida = np.max(salidas_finales) if len(salidas_finales) > 0 else 1
    min_salida = np.min(salidas_finales) if len(salidas_finales) > 0 else 0
    rango_salida = max_salida - min_salida
    margen_salida = rango_salida * 0.2 if rango_salida > 0 else 1
    ax3.set_ylim(min_salida - margen_salida, max_salida + margen_salida)
    
    # Añadir valores
    for i, v in enumerate(salidas_finales):
        offset = rango_salida * 0.05 if rango_salida > 0 else 0.1
        ax3.text(i, v + offset if v > 0 else v - offset, f'{v:.2f}', 
                ha='center', va='bottom' if v > 0 else 'top', fontweight='bold', fontsize=9)
    
    # --- GRÁFICA 4: Mapa de calor de pesos capa oculta ---
    ax4 = fig.add_subplot(gs[2, 0])
    im1 = ax4.imshow(pesos_oculta, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
    ax4.set_xlabel('Entrada', fontsize=11)
    ax4.set_ylabel('Neurona Oculta', fontsize=11)
    ax4.set_title('Pesos Capa Oculta (Entrada → Oculta)', fontsize=13, fontweight='bold', pad=15)
    ax4.set_xticks(range(3))
    ax4.set_yticks(range(3))
    ax4.set_xticklabels(['x0', 'x1', 'x2'])
    ax4.set_yticklabels(['h0', 'h1', 'h2'])
    
    # Añadir valores en el mapa de calor
    for i in range(3):
        for j in range(3):
            ax4.text(j, i, f'{pesos_oculta[i, j]:.2f}', ha='center', va='center', 
                    color='white' if abs(pesos_oculta[i, j]) > 0.5 else 'black', fontsize=9)
    
    plt.colorbar(im1, ax=ax4, label='Valor del Peso')
    
    # --- GRÁFICA 5: Mapa de calor de pesos capa salida ---
    ax5 = fig.add_subplot(gs[2, 1])
    im2 = ax5.imshow(pesos_salida, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
    ax5.set_xlabel('Neurona Oculta', fontsize=11)
    ax5.set_ylabel('Neurona Salida', fontsize=11)
    ax5.set_title('Pesos Capa Salida (Oculta → Salida)', fontsize=13, fontweight='bold', pad=15)
    ax5.set_xticks(range(3))
    ax5.set_yticks(range(2))
    ax5.set_xticklabels(['h0', 'h1', 'h2'])
    ax5.set_yticklabels(['y0', 'y1'])
    
    # Añadir valores en el mapa de calor
    for i in range(2):
        for j in range(3):
            ax5.text(j, i, f'{pesos_salida[i, j]:.2f}', ha='center', va='center', 
                    color='white' if abs(pesos_salida[i, j]) > 0.5 else 'black', fontsize=9)
    
    plt.colorbar(im2, ax=ax5, label='Valor del Peso')
    
    plt.show()
    
    # Información numérica detallada
    print("=" * 70)
    print(" PROPAGACIÓN HACIA ADELANTE (FORWARD PASS)")
    print("=" * 70)
    print(f"\n ENTRADAS: {entradas}")
    print(f"\n CAPA OCULTA:")
    for i in range(3):
        print(f"  h{i} = np.dot({entradas}, {pesos_oculta[i]}) + {sesgos_oculta[i]:.2f}")
        print(f"     = {np.dot(entradas, pesos_oculta[i]):.3f} + {sesgos_oculta[i]:.2f} = {salidas_oculta[i]:.3f}")
    
    print(f"\n CAPA DE SALIDA:")
    for i in range(2):
        print(f"  y{i} = np.dot({salidas_oculta}, {pesos_salida[i]}) + {sesgos_salida[i]:.2f}")
        print(f"     = {np.dot(salidas_oculta, pesos_salida[i]):.3f} + {sesgos_salida[i]:.2f} = {salidas_finales[i]:.3f}")
    
    print(f"\n SALIDAS FINALES: {salidas_finales}")
    print("=" * 70)

# Crear sliders
slider_oculta = widgets.FloatSlider(
    value=1.0, min=-1.0, max=1.0, step=0.05,
    description='Mult. Pesos Oculta:',
    continuous_update=True,
    style={'description_width': '150px'}
)

slider_salida = widgets.FloatSlider(
    value=1.0, min=-1.0, max=1.0, step=0.05,
    description='Mult. Pesos Salida:',
    continuous_update=True,
    style={'description_width': '150px'}
)

# Crear interfaz interactiva
print(" Ajusta los multiplicadores para ver cómo cambian las salidas de la red neuronal\n")
widgets.interact(calcular_y_visualizar,
                mult_oculta=slider_oculta,
                mult_salida=slider_salida)

 Ajusta los multiplicadores para ver cómo cambian las salidas de la red neuronal



interactive(children=(FloatSlider(value=1.0, description='Mult. Pesos Oculta:', max=1.0, min=-1.0, step=0.05, …

<function __main__.calcular_y_visualizar(mult_oculta, mult_salida)>

## Lotes, capas y objetos


Hasta ahora hemos trabajado con un único vector de entrada, es decir, una sola muestra del proceso que queremos modelar.  
Sin embargo, en la práctica resulta mucho más eficiente procesar **varias muestras al mismo tiempo**.  
A este conjunto de muestras se le denomina **lote** (*batch*).

Cada elemento del vector de entrada representa un tipo de dato relevante del sistema que se desea predecir o analizar —por ejemplo, **temperatura**, **voltaje**, **corriente**, **presión**, etc.  
Al agrupar muchas de estas mediciones en un lote, podemos aprovechar las ventajas del procesamiento paralelo, **reduciendo el tiempo de cálculo** y **mejorando la estabilidad del entrenamiento** del modelo.  

Durante el entrenamiento (tema que se abordará más adelante), el uso de lotes permite **ajustar los pesos de las neuronas de manera más precisa**, ya que la actualización de parámetros se basa en la información combinada de múltiples ejemplos.  
Además, mostrar los datos **por lotes pequeños pero representativos** evita que la red memorice los ejemplos (lo que se conoce como *overfitting*) y mejora su capacidad de generalización.

Matemáticamente, esto implica trabajar con **matrices** en lugar de vectores:  
- Cada **fila** de la matriz de entradas representa una muestra (un conjunto de mediciones).  
- Cada **columna** representa una característica o variable (por ejemplo, temperatura, voltaje, corriente...).  
- Los **pesos** también se organizan en una matriz, donde cada **columna de pesos** corresponde a las conexiones hacia una neurona de la capa siguiente.

Por las reglas de la multiplicación de matrices, es necesario **transponer la matriz de pesos** para que las dimensiones coincidan correctamente:  
si la matriz de entradas tiene forma `(n_muestras, n_entradas)` y los pesos `(n_neuronas, n_entradas)`, entonces debemos multiplicar por `pesos.T`.

A continuación, se muestra un ejemplo práctico con NumPy:


In [6]:

import numpy as np

# Supongamos un lote (batch) de 3 muestras, cada una con 4 características
entradas = np.array([
    [1.0, 2.0, 3.0, 2.5],
    [2.0, 5.0, -1.0, 2.0],
    [-1.5, 2.7, 3.3, -0.8]
])

# Definimos una capa con 3 neuronas, cada una con 4 pesos (uno por entrada)
pesos = np.array([
    [0.2, 0.8, -0.5, 1.0],
    [0.5, -0.91, 0.26, -0.5],
    [-0.26, -0.27, 0.17, 0.87]
])

# Bias de cada neurona
bias = np.array([2.0, 3.0, 0.5])

# Cálculo de la salida de la capa para todo el lote
salidas = np.dot(entradas, pesos.T) + bias

print("Salidas de la capa:")
print(salidas)

Salidas de la capa:
[[ 4.8    1.21   2.385]
 [ 8.9   -1.81   0.2  ]
 [ 1.41   1.051  0.026]]


Un ejemplo interactivo en el que se puede ejemplificar mejor el concepto de batches o lotes y el por qué de transponer los datos es el siguiente: 

In [7]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

def visualizar_batches(n_muestras, n_neuronas):
    """Visualiza el procesamiento de batches con operaciones matriciales"""
    
    # Configuración fija
    n_entradas = 4
    
    # Generar datos de ejemplo
    np.random.seed(42)
    entradas = np.random.randn(n_muestras, n_entradas) * 2
    pesos = np.random.randn(n_neuronas, n_entradas)
    bias = np.random.randn(n_neuronas)
    
    # Calcular salidas
    salidas = np.dot(entradas, pesos.T) + bias
    
    # Simular tiempos de procesamiento (valores más realistas)
    # Tiempo base por operación (unidades arbitrarias)
    tiempo_base_por_muestra = 10.0
    
    # Tiempo procesando muestra por muestra (lineal con el número de muestras)
    tiempo_individual = tiempo_base_por_muestra * n_muestras
    
    # Tiempo procesando en batch (con overhead fijo + escalado sublineal)
    overhead_batch = 3.0  # Overhead inicial de preparar el batch
    tiempo_batch = overhead_batch + (tiempo_base_por_muestra * n_muestras * 0.3)
    
    # Cerrar figuras previas
    plt.close('all')
    
    # Crear visualización
    fig = plt.figure(figsize=(16, 12))
    gs = fig.add_gridspec(4, 3, hspace=0.45, wspace=0.35)
    
    # --- GRÁFICA 1: Matriz de Entradas ---
    ax1 = fig.add_subplot(gs[0, 0])
    im1 = ax1.imshow(entradas, cmap='viridis', aspect='auto')
    ax1.set_title(f'Matriz de Entradas\n({n_muestras} muestras x {n_entradas} caracteristicas)', 
                  fontsize=11, fontweight='bold', pad=10)
    ax1.set_xlabel('Caracteristicas (features)', fontsize=9)
    ax1.set_ylabel('Muestras (samples)', fontsize=9)
    ax1.set_xticks(range(n_entradas))
    ax1.set_yticks(range(n_muestras))
    ax1.set_xticklabels([f'x{i}' for i in range(n_entradas)], fontsize=8)
    ax1.set_yticklabels([f'M{i}' for i in range(n_muestras)], fontsize=8)
    
    # Añadir valores en la matriz
    for i in range(n_muestras):
        for j in range(n_entradas):
            color = 'white' if abs(entradas[i, j]) > 1 else 'black'
            ax1.text(j, i, f'{entradas[i, j]:.1f}', ha='center', va='center', 
                    color=color, fontsize=7)
    
    plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
    
    # --- GRÁFICA 2: Matriz de Pesos ---
    ax2 = fig.add_subplot(gs[0, 1])
    im2 = ax2.imshow(pesos, cmap='coolwarm', aspect='auto', vmin=-2, vmax=2)
    ax2.set_title(f'Matriz de Pesos\n({n_neuronas} neuronas x {n_entradas} pesos)', 
                  fontsize=11, fontweight='bold', pad=10)
    ax2.set_xlabel('Pesos por entrada', fontsize=9)
    ax2.set_ylabel('Neuronas', fontsize=9)
    ax2.set_xticks(range(n_entradas))
    ax2.set_yticks(range(n_neuronas))
    ax2.set_xticklabels([f'w{i}' for i in range(n_entradas)], fontsize=8)
    ax2.set_yticklabels([f'N{i}' for i in range(n_neuronas)], fontsize=8)
    
    # Añadir valores en la matriz
    for i in range(n_neuronas):
        for j in range(n_entradas):
            color = 'white' if abs(pesos[i, j]) > 1 else 'black'
            ax2.text(j, i, f'{pesos[i, j]:.1f}', ha='center', va='center', 
                    color=color, fontsize=7)
    
    plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
    
    # --- GRÁFICA 3: Matriz de Pesos Transpuesta ---
    ax3 = fig.add_subplot(gs[0, 2])
    im3 = ax3.imshow(pesos.T, cmap='coolwarm', aspect='auto', vmin=-2, vmax=2)
    ax3.set_title(f'Matriz de Pesos Transpuesta\n({n_entradas} x {n_neuronas})', 
                  fontsize=11, fontweight='bold', pad=10)
    ax3.set_xlabel('Neuronas', fontsize=9)
    ax3.set_ylabel('Pesos', fontsize=9)
    ax3.set_xticks(range(n_neuronas))
    ax3.set_yticks(range(n_entradas))
    ax3.set_xticklabels([f'N{i}' for i in range(n_neuronas)], fontsize=8)
    ax3.set_yticklabels([f'w{i}' for i in range(n_entradas)], fontsize=8)
    
    # Añadir valores en la matriz
    for i in range(n_entradas):
        for j in range(n_neuronas):
            color = 'white' if abs(pesos.T[i, j]) > 1 else 'black'
            ax3.text(j, i, f'{pesos.T[i, j]:.1f}', ha='center', va='center', 
                    color=color, fontsize=7)
    
    plt.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)
    
    # --- GRÁFICA 4: Diagrama de operación ---
    ax4 = fig.add_subplot(gs[1, :])
    ax4.set_xlim(0, 10)
    ax4.set_ylim(0, 3)
    ax4.axis('off')
    ax4.set_title('Operacion Matricial: Entradas x Pesos.T + Bias', 
                  fontsize=13, fontweight='bold', pad=15)
    
    # Dibujar las operaciones
    ax4.text(1, 1.5, f'Entradas\n[{n_muestras}x{n_entradas}]', ha='center', va='center', 
            fontsize=10, bbox=dict(boxstyle='round', facecolor='#88D498', alpha=0.7))
    ax4.text(2.5, 1.5, 'x', ha='center', va='center', fontsize=16, fontweight='bold')
    ax4.text(4, 1.5, f'Pesos.T\n[{n_entradas}x{n_neuronas}]', ha='center', va='center', 
            fontsize=10, bbox=dict(boxstyle='round', facecolor='#FF6B9D', alpha=0.7))
    ax4.text(5.5, 1.5, '+', ha='center', va='center', fontsize=16, fontweight='bold')
    ax4.text(7, 1.5, f'Bias\n[{n_neuronas}]', ha='center', va='center', 
            fontsize=10, bbox=dict(boxstyle='round', facecolor='#C5A3FF', alpha=0.7))
    ax4.text(8.3, 1.5, '=', ha='center', va='center', fontsize=16, fontweight='bold')
    ax4.text(9.3, 1.5, f'Salidas\n[{n_muestras}x{n_neuronas}]', ha='center', va='center', 
            fontsize=10, bbox=dict(boxstyle='round', facecolor='#FFD93D', alpha=0.7))
    
    # Explicación
    ax4.text(5, 0.3, 'Todas las muestras se procesan en paralelo', 
            ha='center', fontsize=11, style='italic', color='#2D5016', fontweight='bold')
    
    # --- GRÁFICA 5: Matriz de Salidas ---
    ax5 = fig.add_subplot(gs[2, 0])
    im5 = ax5.imshow(salidas, cmap='plasma', aspect='auto')
    ax5.set_title(f'Matriz de Salidas\n({n_muestras} muestras x {n_neuronas} neuronas)', 
                  fontsize=11, fontweight='bold', pad=10)
    ax5.set_xlabel('Neuronas', fontsize=9)
    ax5.set_ylabel('Muestras', fontsize=9)
    ax5.set_xticks(range(n_neuronas))
    ax5.set_yticks(range(n_muestras))
    ax5.set_xticklabels([f'N{i}' for i in range(n_neuronas)], fontsize=8)
    ax5.set_yticklabels([f'M{i}' for i in range(n_muestras)], fontsize=8)
    
    # Añadir valores en la matriz
    for i in range(n_muestras):
        for j in range(n_neuronas):
            color = 'white' if abs(salidas[i, j]) > 2 else 'black'
            ax5.text(j, i, f'{salidas[i, j]:.1f}', ha='center', va='center', 
                    color=color, fontsize=7)
    
    plt.colorbar(im5, ax=ax5, fraction=0.046, pad=0.04)
    
    # --- GRÁFICA 6: Salidas por muestra ---
    ax6 = fig.add_subplot(gs[2, 1:])
    x = np.arange(n_neuronas)
    width = 0.8 / n_muestras
    colores = plt.cm.Set3(np.linspace(0, 1, n_muestras))
    
    for i in range(n_muestras):
        offset = (i - n_muestras/2) * width + width/2
        ax6.bar(x + offset, salidas[i], width, label=f'Muestra {i}', 
               color=colores[i], alpha=0.8)
    
    ax6.set_xlabel('Neurona', fontsize=10)
    ax6.set_ylabel('Valor de Salida', fontsize=10)
    ax6.set_title('Salidas de cada Muestra por Neurona', fontsize=11, fontweight='bold', pad=10)
    ax6.set_xticks(x)
    ax6.set_xticklabels([f'N{i}' for i in range(n_neuronas)])
    ax6.legend(fontsize=8, loc='best')
    ax6.grid(axis='y', alpha=0.3)
    ax6.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
    
    # --- GRÁFICA 7: Comparación de eficiencia ---
    ax7 = fig.add_subplot(gs[3, :])
    
    metodos = ['Muestra por Muestra\n(Secuencial)', 'Procesamiento en Batch\n(Paralelo)']
    tiempos = [tiempo_individual, tiempo_batch]
    colores_barras = ['#FF6B6B', '#4ECDC4']
    
    y_pos = np.arange(len(metodos))
    bars = ax7.barh(y_pos, tiempos, color=colores_barras, alpha=0.7, height=0.5)
    
    ax7.set_yticks(y_pos)
    ax7.set_yticklabels(metodos, fontsize=10)
    ax7.set_xlabel('Tiempo de Procesamiento (unidades arbitrarias)', fontsize=10)
    ax7.set_title('Comparacion de Eficiencia: Secuencial vs Batch', 
                  fontsize=12, fontweight='bold', pad=15)
    ax7.grid(axis='x', alpha=0.3)
    
    # Establecer límites apropiados
    max_tiempo = max(tiempos)
    ax7.set_xlim([0, max_tiempo * 1.25])
    
    # Añadir valores en las barras
    for i, (bar, tiempo) in enumerate(zip(bars, tiempos)):
        ax7.text(tiempo + max_tiempo * 0.02, bar.get_y() + bar.get_height()/2, 
                f'{tiempo:.1f}', va='center', ha='left', fontweight='bold', fontsize=10)
    
    # Calcular y mostrar speedup
    speedup = tiempo_individual / tiempo_batch if tiempo_batch > 0 else 1
    
    # Texto del speedup con explicación según el caso
    if n_muestras == 1:
        speedup_text = f'Con 1 muestra: Batch tiene overhead inicial (~{overhead_batch:.0f} unidades)\nPero escala mejor con mas muestras!'
    else:
        speedup_text = f'Speedup: {speedup:.1f}x mas rapido con batches!\nProcesar {n_muestras} muestras en paralelo es mucho mas eficiente'
    
    ax7.text(max_tiempo * 0.5, -0.85, speedup_text,
            ha='center', va='top', fontsize=10, fontweight='bold', 
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    plt.show()
    
    # Información numérica
    print("=" * 80)
    print("RESUMEN DE OPERACIONES MATRICIALES CON BATCHES")
    print("=" * 80)
    print(f"\nDIMENSIONES:")
    print(f"   Entradas:     {entradas.shape} = ({n_muestras} muestras, {n_entradas} caracteristicas)")
    print(f"   Pesos:        {pesos.shape} = ({n_neuronas} neuronas, {n_entradas} pesos c/u)")
    print(f"   Pesos.T:      {pesos.T.shape} = ({n_entradas} caracteristicas, {n_neuronas} neuronas)")
    print(f"   Bias:         {bias.shape} = ({n_neuronas} valores)")
    print(f"   Salidas:      {salidas.shape} = ({n_muestras} muestras, {n_neuronas} neuronas)")
    
    print(f"\nOPERACION:")
    print(f"   salidas = np.dot(entradas, pesos.T) + bias")
    print(f"   salidas = np.dot({entradas.shape}, {pesos.T.shape}) + {bias.shape}")
    print(f"   salidas = {salidas.shape}")
    
    print(f"\nEFICIENCIA:")
    print(f"   Tiempo secuencial (muestra por muestra): {tiempo_individual:.1f} unidades")
    print(f"   Tiempo batch (procesamiento paralelo):   {tiempo_batch:.1f} unidades")
    print(f"   Speedup:                                   {speedup:.2f}x")
    
    if n_muestras == 1:
        print(f"\n   NOTA: Con 1 muestra, el batch tiene overhead inicial.")
        print(f"         La ventaja aparece con multiples muestras (prueba con 3-5 muestras)!")
    
    print(f"\nINTERPRETACION:")
    print(f"   - Cada FILA de 'entradas' = 1 muestra con {n_entradas} caracteristicas")
    print(f"   - Cada COLUMNA de 'entradas' = 1 caracteristica medida en {n_muestras} muestras")
    print(f"   - Cada FILA de 'salidas' = salida de las {n_neuronas} neuronas para 1 muestra")
    print(f"   - Se procesan {n_muestras} muestras simultaneamente (paraleizacion)")
    
    print(f"\nEJEMPLO DE SALIDA (Muestra 0):")
    print(f"   Entrada: {entradas[0]}")
    print(f"   Salida:  {salidas[0]}")
    
    print("=" * 80)

# Crear sliders
slider_muestras = widgets.IntSlider(
    value=3, min=1, max=5, step=1,
    description='Num. Muestras:',
    continuous_update=False,
    style={'description_width': '120px'}
)

slider_neuronas = widgets.IntSlider(
    value=3, min=2, max=5, step=1,
    description='Num. Neuronas:',
    continuous_update=False,
    style={'description_width': '120px'}
)

# Crear interfaz interactiva
print("Ajusta el tamano del batch y numero de neuronas para ver las operaciones matriciales\n")
widgets.interact(visualizar_batches,
                n_muestras=slider_muestras,
                n_neuronas=slider_neuronas)

Ajusta el tamano del batch y numero de neuronas para ver las operaciones matriciales



interactive(children=(IntSlider(value=3, continuous_update=False, description='Num. Muestras:', max=5, min=1, …

<function __main__.visualizar_batches(n_muestras, n_neuronas)>

- Programación Orientada a Objetos (POO) para Redes Neuronales

Hasta este punto, hemos implementado el cálculo de la salida de una capa de forma manual.  
Sin embargo, a medida que se agregan más capas o se modifican las dimensiones de entrada y salida, el código puede volverse repetitivo y difícil de mantener.  

Para resolver este problema, es útil aplicar los principios de la **Programación Orientada a Objetos (POO)**.  
Con la POO podemos definir una **clase `Layer_Dense`** que represente una capa densa (totalmente conectada) y que almacene tanto sus **pesos**, **biases** como el método que calcula su **propagación hacia adelante** (*forward pass*).  

De este modo, podemos crear fácilmente varias capas con diferentes tamaños y conectarlas entre sí, sin tener que reescribir el mismo código cada vez.  
Esto permite construir redes neuronales más profundas simplemente **instanciando nuevas capas** y pasándoles la salida de la capa anterior como entrada.

A continuación, se muestra el código completo con NumPy, implementando dos capas densas conectadas secuencialmente:

In [8]:
import numpy as np

# Fijamos la semilla aleatoria para reproducibilidad
np.random.seed(0)

# Definimos una matriz de entrada X (3 muestras, 4 características cada una)
X = [
    [1.0, 2.0, 3.0, 2.5],
    [2.0, 5.0, -1.0, 2.0],
    [-1.5, 2.7, 3.3, -0.8]
]

# Definición de la clase Layer_Dense
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        # Inicializamos los pesos con valores aleatorios pequeños
        # np.random.randn(n_inputs, n_neurons) genera una matriz de dimensiones
        # [n_inputs x n_neurons], donde cada columna corresponde a los pesos de una neurona.
        # Por eso ya NO es necesario usar la transpuesta al multiplicar.
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        
        # Inicializamos los biases en ceros (uno por neurona)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        # Calcula la salida de la capa como: entradas * pesos + bias
        # np.dot realiza la multiplicación matricial entre inputs y weights
        self.output = np.dot(inputs, self.weights) + self.biases

# Creamos dos capas densas:
# - La primera recibe 4 entradas y tiene 5 neuronas.
# - La segunda recibe 5 entradas (las salidas de la anterior) y produce 2 salidas.
layer1 = Layer_Dense(4, 5)
layer2 = Layer_Dense(5, 2)

# Pasamos la entrada X por la primera capa
layer1.forward(X)

# Pasamos la salida de la primera capa como entrada a la segunda
layer2.forward(layer1.output)

# Imprimimos los resultados
print("Salida de la primera capa:")
print(layer1.output)

print("\nSalida de la segunda capa:")
print(layer2.output)


Salida de la primera capa:
[[ 0.10758131  1.03983522  0.24462411  0.31821498  0.18851053]
 [-0.08349796  0.70846411  0.00293357  0.44701525  0.36360538]
 [-0.50763245  0.55688422  0.07987797 -0.34889573  0.04553042]]

Salida de la segunda capa:
[[ 0.148296   -0.08397602]
 [ 0.14100315 -0.01340469]
 [ 0.20124979 -0.07290616]]


## Funciones de activacion

En una red neuronal, **la función de activación** es la operación que se aplica a la salida de cada neurona **después del cálculo lineal** (es decir, después del producto entre las entradas y los pesos más el bias).  

Su propósito es **introducir no linealidad** en el modelo, permitiendo que la red aprenda relaciones complejas entre las variables de entrada y salida.  

Si solo utilizáramos operaciones lineales (multiplicaciones y sumas), sin una función de activación, toda la red se comportaría como una **única transformación lineal**, sin importar cuántas capas tuviera.  

En otras palabras, sin funciones de activación, la red **no podría aproximar funciones no lineales** ni capturar patrones complejos en los datos.

De forma intuitiva, las funciones de activación pueden verse como un mecanismo que **decide qué neuronas "se disparan" o permanecen inactivas**, similar al comportamiento de las neuronas biológicas del cerebro.

### Tipos comunes de funciones de activación

1. **Función Step (Escalón)**  
   Es la más simple. Devuelve 1 si la entrada es positiva, y 0 en caso contrario.  
   Se usó en los primeros modelos neuronales (como el perceptrón), pero casi no se utiliza hoy en día debido a que no permite un ajuste gradual ni continuidad en la salida.
   
   $$
   f(x) =
   \begin{cases}
   1, & \text{si } x \ge 0 \\
   0, & \text{si } x < 0
   \end{cases}
   $$

2. **Función Sigmoide**  
   Convierte cualquier valor real en un número entre 0 y 1, con una forma suave y continua.  
   Se usa principalmente en capas de salida cuando se requiere interpretar la salida como una probabilidad.
   
   $$
   f(x) = \frac{1}{1 + e^{-x}}
   $$

3. **Función ReLU (*Rectified Linear Unit*)**  
   Es la función más popular para las **capas ocultas** debido a su sencillez y efectividad.  
   Devuelve el valor de entrada si es positivo y 0 si es negativo, permitiendo que el modelo mantenga una parte de la información (granularidad) mientras descarta valores negativos.
   
   $$
   f(x) = \max(0, x)
   $$

En general, distintas capas pueden tener **distintas funciones de activación** dependiendo de la tarea.  
Por ejemplo, las capas ocultas suelen usar ReLU, mientras que las capas de salida pueden usar *softmax* o *sigmoid* según el tipo de problema (clasificación o regresión).  
Este tema se tratará en mayor profundidad más adelante.

A continuación, se muestra un ejemplo práctico en Python implementando la función ReLU sobre una capa neuronal:

In [24]:
import numpy as np

# Fijamos la semilla para reproducibilidad

np.random.seed(0)

# Entradas: 3 muestras con 4 características cada una

X = [

    [1.0, 2.0, 3.0, 2.5],

    [2.0, 5.0, -1.0, 2.0],

    [-1.5, 2.7, 3.3, -0.8]

]

# Definición de una capa densa (como antes)

class Layer_Dense:

    def __init__(self, n_inputs, n_neurons):

        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)

        self.biases = np.zeros((1, n_neurons))

    

    def forward(self, inputs):

        self.output = np.dot(inputs, self.weights) + self.biases

# Definición de la función ReLU

class Activation_ReLU:

    def forward(self, inputs):

        # np.maximum aplica ReLU elemento a elemento

        self.output = np.maximum(0, inputs)

# Creamos una capa densa de 4 entradas y 5 neuronas

layer1 = Layer_Dense(4, 5)

# Calculamos su salida

layer1.forward(X)

# Aplicamos la activación ReLU

activation1 = Activation_ReLU()

activation1.forward(layer1.output)

# Mostramos resultados

print("Salida lineal de la capa:")

print(layer1.output)

print("\nSalida después de aplicar ReLU:")

print(activation1.output)

Salida lineal de la capa:
[[ 0.10758131  1.03983522  0.24462411  0.31821498  0.18851053]
 [-0.08349796  0.70846411  0.00293357  0.44701525  0.36360538]
 [-0.50763245  0.55688422  0.07987797 -0.34889573  0.04553042]]

Salida después de aplicar ReLU:
[[0.10758131 1.03983522 0.24462411 0.31821498 0.18851053]
 [0.         0.70846411 0.00293357 0.44701525 0.36360538]
 [0.         0.55688422 0.07987797 0.         0.04553042]]


Un ejemplo más interactivo es el siguiente:

In [23]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

# Definir las funciones de activación
def step_function(x):
    """Función Step (Escalón)"""
    return np.where(x >= 0, 1, 0)

def sigmoid_function(x):
    """Función Sigmoide"""
    return 1 / (1 + np.exp(-x))

def relu_function(x):
    """Función ReLU"""
    return np.maximum(0, x)

# Clases del ejemplo original
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, seed=0):
        np.random.seed(seed)
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

def visualizar_activaciones(mult_pesos, seed):
    """Visualiza el efecto de diferentes funciones de activación"""
    
    # Datos de entrada (3 muestras, 4 características)
    X = np.array([
        [1.0, 2.0, 3.0, 2.5],
        [2.0, 5.0, -1.0, 2.0],
        [-1.5, 2.7, 3.3, -0.8]
    ])
    
    # Crear capa densa
    layer1 = Layer_Dense(4, 5, seed=seed)
    layer1.weights = layer1.weights * mult_pesos  # Aplicar multiplicador
    layer1.forward(X)
    
    # Salida lineal (sin activación)
    salida_lineal = layer1.output
    
    # Aplicar diferentes funciones de activación
    salida_step = step_function(salida_lineal)
    salida_sigmoid = sigmoid_function(salida_lineal)
    salida_relu = relu_function(salida_lineal)
    
    # Crear figura
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.4, wspace=0.3)
    
    # --- FILA 1: Gráficas de las funciones de activación ---
    x_range = np.linspace(-5, 5, 1000)
    
    # Step
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(x_range, step_function(x_range), linewidth=2.5, color='#FF6B6B')
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3)
    ax1.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3)
    ax1.grid(alpha=0.3)
    ax1.set_xlabel('x', fontsize=10)
    ax1.set_ylabel('f(x)', fontsize=10)
    ax1.set_title('Funcion Step (Escalon)', fontsize=11, fontweight='bold', pad=10)
    ax1.set_ylim(-0.2, 1.2)
    ax1.text(2.5, 0.5, 'f(x) = 1 si x >= 0\nf(x) = 0 si x < 0', 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), fontsize=8)
    
    # Sigmoid
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(x_range, sigmoid_function(x_range), linewidth=2.5, color='#4ECDC4')
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3)
    ax2.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3)
    ax2.axhline(y=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.5)
    ax2.grid(alpha=0.3)
    ax2.set_xlabel('x', fontsize=10)
    ax2.set_ylabel('f(x)', fontsize=10)
    ax2.set_title('Funcion Sigmoide', fontsize=11, fontweight='bold', pad=10)
    ax2.set_ylim(-0.2, 1.2)
    ax2.text(2.5, 0.5, 'f(x) = 1 / (1 + e^(-x))', 
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5), fontsize=8)
    
    # ReLU
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.plot(x_range, relu_function(x_range), linewidth=2.5, color='#45B7D1')
    ax3.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3)
    ax3.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3)
    ax3.grid(alpha=0.3)
    ax3.set_xlabel('x', fontsize=10)
    ax3.set_ylabel('f(x)', fontsize=10)
    ax3.set_title('Funcion ReLU', fontsize=11, fontweight='bold', pad=10)
    ax3.text(2.5, 2.5, 'f(x) = max(0, x)', 
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5), fontsize=8)
    
    # --- FILA 2: Salida lineal y comparación ---
    
    # Salida lineal (antes de activación)
    ax4 = fig.add_subplot(gs[1, :])
    n_neuronas = salida_lineal.shape[1]
    n_muestras = salida_lineal.shape[0]
    x_pos = np.arange(n_neuronas)
    width = 0.25
    
    for i in range(n_muestras):
        offset = (i - 1) * width
        ax4.bar(x_pos + offset, salida_lineal[i], width, 
               label=f'Muestra {i}', alpha=0.8)
    
    ax4.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax4.set_xlabel('Neurona', fontsize=10)
    ax4.set_ylabel('Valor de salida', fontsize=10)
    ax4.set_title('Salida Lineal (Antes de Activacion)', fontsize=12, fontweight='bold', pad=10)
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels([f'N{i}' for i in range(n_neuronas)])
    ax4.legend(fontsize=8, loc='best')
    ax4.grid(axis='y', alpha=0.3)
    
    # --- FILA 3: Comparación de salidas con diferentes activaciones ---
    
    # Step
    ax5 = fig.add_subplot(gs[2, 0])
    im5 = ax5.imshow(salida_step, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    ax5.set_title('Salida con Step', fontsize=11, fontweight='bold', pad=10)
    ax5.set_xlabel('Neurona', fontsize=9)
    ax5.set_ylabel('Muestra', fontsize=9)
    ax5.set_xticks(range(n_neuronas))
    ax5.set_yticks(range(n_muestras))
    ax5.set_xticklabels([f'N{i}' for i in range(n_neuronas)], fontsize=8)
    ax5.set_yticklabels([f'M{i}' for i in range(n_muestras)], fontsize=8)
    
    for i in range(n_muestras):
        for j in range(n_neuronas):
            ax5.text(j, i, f'{salida_step[i, j]:.2f}', ha='center', va='center', 
                    color='black', fontsize=8, fontweight='bold')
    
    plt.colorbar(im5, ax=ax5, fraction=0.046, pad=0.04)
    
    # Sigmoid
    ax6 = fig.add_subplot(gs[2, 1])
    im6 = ax6.imshow(salida_sigmoid, cmap='viridis', aspect='auto', vmin=0, vmax=1)
    ax6.set_title('Salida con Sigmoid', fontsize=11, fontweight='bold', pad=10)
    ax6.set_xlabel('Neurona', fontsize=9)
    ax6.set_ylabel('Muestra', fontsize=9)
    ax6.set_xticks(range(n_neuronas))
    ax6.set_yticks(range(n_muestras))
    ax6.set_xticklabels([f'N{i}' for i in range(n_neuronas)], fontsize=8)
    ax6.set_yticklabels([f'M{i}' for i in range(n_muestras)], fontsize=8)
    
    for i in range(n_muestras):
        for j in range(n_neuronas):
            color = 'white' if salida_sigmoid[i, j] > 0.5 else 'black'
            ax6.text(j, i, f'{salida_sigmoid[i, j]:.2f}', ha='center', va='center', 
                    color=color, fontsize=8, fontweight='bold')
    
    plt.colorbar(im6, ax=ax6, fraction=0.046, pad=0.04)
    
    # ReLU
    ax7 = fig.add_subplot(gs[2, 2])
    im7 = ax7.imshow(salida_relu, cmap='plasma', aspect='auto')
    ax7.set_title('Salida con ReLU', fontsize=11, fontweight='bold', pad=10)
    ax7.set_xlabel('Neurona', fontsize=9)
    ax7.set_ylabel('Muestra', fontsize=9)
    ax7.set_xticks(range(n_neuronas))
    ax7.set_yticks(range(n_muestras))
    ax7.set_xticklabels([f'N{i}' for i in range(n_neuronas)], fontsize=8)
    ax7.set_yticklabels([f'M{i}' for i in range(n_muestras)], fontsize=8)
    
    for i in range(n_muestras):
        for j in range(n_neuronas):
            color = 'white' if salida_relu[i, j] > 0.5 else 'black'
            ax7.text(j, i, f'{salida_relu[i, j]:.2f}', ha='center', va='center', 
                    color=color, fontsize=8, fontweight='bold')
    
    plt.colorbar(im7, ax=ax7, fraction=0.046, pad=0.04)
    
    plt.show()
    
    # Información detallada
    print("=" * 80)
    print("COMPARACION DE FUNCIONES DE ACTIVACION")
    print("=" * 80)
    print(f"\nSALIDA LINEAL (sin activacion):")
    print(salida_lineal)
    print(f"\nDespues de STEP:")
    print(salida_step)
    print(f"  - Valores binarios: {np.unique(salida_step)}")
    print(f"  - Neuronas activas: {np.sum(salida_step)} de {salida_step.size}")
    print(f"\nDespues de SIGMOID:")
    print(salida_sigmoid)
    print(f"  - Rango: [{np.min(salida_sigmoid):.3f}, {np.max(salida_sigmoid):.3f}]")
    print(f"  - Promedio: {np.mean(salida_sigmoid):.3f}")
    print(f"\nDespues de RELU:")
    print(salida_relu)
    print(f"  - Valores negativos convertidos a 0")
    print(f"  - Neuronas muertas (salida=0): {np.sum(salida_relu == 0)} de {salida_relu.size}")
    print(f"  - Rango activo: [0, {np.max(salida_relu):.3f}]")
    
    print("\n" + "=" * 80)
    print("CARACTERISTICAS PRINCIPALES:")
    print("=" * 80)
    print("STEP:    - Binaria (0 o 1)")
    print("         - No diferenciable en x=0")
    print("         - Perdida de informacion granular")
    print("\nSIGMOID: - Suave y continua")
    print("         - Salida entre 0 y 1")
    print("         - Util para probabilidades")
    print("         - Problema: vanishing gradient")
    print("\nRELU:    - Simple y eficiente")
    print("         - Mantiene valores positivos intactos")
    print("         - Elimina valores negativos")
    print("         - Mas usado en capas ocultas")
    print("         - Problema: neuronas muertas")
    print("=" * 80)

# Crear sliders
slider_pesos = widgets.FloatSlider(
    value=1.0, min=0.1, max=3.0, step=0.1,
    description='Mult. Pesos:',
    continuous_update=False,
    style={'description_width': '100px'}
)

slider_seed = widgets.IntSlider(
    value=0, min=0, max=10, step=1,
    description='Semilla:',
    continuous_update=False,
    style={'description_width': '100px'}
)

# Crear interfaz interactiva
print("Ajusta los parametros para ver como las funciones de activacion transforman las salidas\n")
widgets.interact(visualizar_activaciones,
                mult_pesos=slider_pesos,
                seed=slider_seed)

Ajusta los parametros para ver como las funciones de activacion transforman las salidas



interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Mult. Pesos:', max=3.0, min…

<function __main__.visualizar_activaciones(mult_pesos, seed)>

## Activacion softmax

## Función de Activación Softmax

Hasta ahora hemos visto funciones como la **ReLU**, que ayudan a las *hidden layers* a introducir no linealidad en los cálculos.  

Sin embargo, cuando llegamos a la **última capa** (la capa de salida), muchas veces queremos **interpretar los resultados de las neuronas como probabilidades**.  

Por ejemplo, si una red debe clasificar una entrada en tres clases (gato, perro, pez), no queremos simplemente tres números sin escala, sino algo que nos diga *qué tan probable* es que la entrada pertenezca a cada clase.  

Ahí entra **Softmax**.


### ¿Qué hace Softmax?

Softmax toma los valores de salida de las neuronas (a menudo llamados *logits*) y los convierte en **valores entre 0 y 1**, que además **suman exactamente 1**.  

Eso significa que podemos interpretarlos como **probabilidades normalizadas**.

Matemáticamente:

$$
\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j} e^{z_j}}
$$



### ¿Por qué se usa en la salida?

- Porque **permite comparar las salidas de las neuronas** en una misma escala.  
- Facilita el cálculo del **error** (que veremos más adelante con *cross-entropy*).  
- En clasificación, cada neurona puede representar una clase distinta, y Softmax nos da la probabilidad asociada a cada una.



### Prevención de overflow

El término $e^{z_i}$ puede crecer muy rápido si $z_i$ es grande.  

Para evitar desbordamientos numéricos, restamos el máximo valor de $z$ antes de aplicar la exponencial (esto **no cambia el resultado**, solo lo estabiliza):

$$
\text{Softmax}(z_i) = \frac{e^{z_i - \max(z)}}{\sum_{j} e^{z_j - \max(z)}}
$$



### En resumen

Softmax:
- Normaliza las salidas.  
- Permite interpretarlas como probabilidades.  
- Se usa casi siempre en la **última capa de redes de clasificación**.  



## Ejemplo en Python

A continuación implementamos Softmax como una clase de activación, integrada al esquema que venimos usando con las capas densas.

In [11]:
import numpy as np

# Datos de entrada (3 muestras, 4 características cada una)
X = [[1.0, 2.0, 3.0, 2.5],
     [2.0, 5.0, -1.0, 2.0],
     [-1.5, 2.7, 3.3, -0.8]]

# Fijar semilla para reproducibilidad
np.random.seed(0)

# Capa densa general
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        # Pesos aleatorios pequeños para estabilidad numérica
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        # Biases inicializados en cero
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        # Producto punto entre entradas y pesos + biases
        self.output = np.dot(inputs, self.weights) + self.biases

# Función de activación ReLU
class Activation_ReLU:
    def forward(self, inputs):
        # Aplica ReLU: deja pasar positivos, corta negativos
        self.output = np.maximum(0, inputs)

# Función de activación Softmax
class Activation_Softmax:
    def forward(self, inputs):
        # Restar el máximo para evitar overflow numérico
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        # Normalizar dividiendo por la suma de exponentiales (por fila)
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

# Capa oculta (4 entradas → 5 neuronas)
layer1 = Layer_Dense(4, 5)
activation1 = Activation_ReLU()

# Capa de salida (5 entradas → 3 neuronas)
layer2 = Layer_Dense(5, 3)
activation2 = Activation_Softmax()

# Paso hacia adelante
layer1.forward(X)
activation1.forward(layer1.output)

layer2.forward(activation1.output)
activation2.forward(layer2.output)

# Mostrar las probabilidades resultantes
print("Salidas después de Softmax:")
print(activation2.output)


Salidas después de Softmax:
[[0.30326886 0.40108669 0.29564444]
 [0.32834771 0.36829707 0.30335522]
 [0.31750561 0.37292873 0.30956567]]



Todas las filas suman 1, lo que significa que Softmax normalizó correctamente las activaciones.

A continuación se presenta un ejemplo interactivo acerca de la implementación de Softmax

In [18]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyBboxPatch

def visualizar_softmax(temperatura, magnitud_logits):
    """Visualiza el proceso de Softmax paso a paso"""
    
    # Generar logits de ejemplo (3 clases)
    np.random.seed(42)
    logits_base = np.array([2.0, 1.0, 0.1])
    logits = logits_base * magnitud_logits
    
    # Aplicar Softmax con temperatura
    def softmax_con_temp(z, T=1.0):
        z_scaled = z / T
        # Prevención de overflow
        exp_values = np.exp(z_scaled - np.max(z_scaled))
        return exp_values / np.sum(exp_values)
    
    probabilidades = softmax_con_temp(logits, temperatura)
    
    # Calcular valores intermedios para visualización
    logits_shifted = logits - np.max(logits)
    exp_values = np.exp(logits_shifted / temperatura)
    suma_exp = np.sum(exp_values)
    
    # Crear figura
    fig = plt.figure(figsize=(18, 13))
    gs = fig.add_gridspec(4, 3, hspace=0.6, wspace=0.4, top=0.96, bottom=0.04)
    
    clases = ['Gato', 'Perro', 'Pez']
    colores = ['#FF6B6B', '#4ECDC4', '#95E1D3']
    
    # --- GRAFICA 1: Logits de entrada ---
    ax1 = fig.add_subplot(gs[0, 0])
    bars1 = ax1.bar(range(3), logits, color=colores, alpha=0.7, edgecolor='black', linewidth=2)
    ax1.set_title('Paso 1: Logits (Salida Cruda)', fontsize=12, fontweight='bold', pad=10)
    ax1.set_ylabel('Valor', fontsize=10)
    ax1.set_xticks(range(3))
    ax1.set_xticklabels(clases, fontsize=10)
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
    ax1.grid(axis='y', alpha=0.3)
    
    # Ajustar ylim para dar espacio a los textos
    y_range = max(logits) - min(logits)
    ax1.set_ylim([min(logits) - y_range*0.4, max(logits) + y_range*0.4])
    
    for i, (bar, val) in enumerate(zip(bars1, logits)):
        height = bar.get_height()
        offset = y_range*0.08 if height > 0 else -y_range*0.08
        ax1.text(bar.get_x() + bar.get_width()/2, height + offset, 
                f'{val:.2f}', ha='center', va='bottom' if height > 0 else 'top',
                fontweight='bold', fontsize=10)
    
    # --- GRAFICA 2: Exponenciales ---
    ax2 = fig.add_subplot(gs[0, 1])
    bars2 = ax2.bar(range(3), exp_values, color=colores, alpha=0.7, edgecolor='black', linewidth=2)
    ax2.set_title('Paso 2: Exponenciales exp(z)', fontsize=12, fontweight='bold', pad=10)
    ax2.set_ylabel('Valor', fontsize=10)
    ax2.set_xticks(range(3))
    ax2.set_xticklabels(clases, fontsize=10)
    ax2.grid(axis='y', alpha=0.3)
    
    # Ajustar ylim para dar espacio
    ax2.set_ylim([0, max(exp_values) * 1.35])
    
    for i, (bar, val) in enumerate(zip(bars2, exp_values)):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(exp_values)*0.05, 
                f'{val:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Mostrar suma (posicionada arriba sin solaparse)
    ax2.text(1, max(exp_values)*1.2, f'Suma = {suma_exp:.2f}', 
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7, pad=0.5),
            fontsize=10, fontweight='bold', ha='center', va='center')
    
    # --- GRAFICA 3: Probabilidades finales ---
    ax3 = fig.add_subplot(gs[0, 2])
    bars3 = ax3.bar(range(3), probabilidades, color=colores, alpha=0.7, 
                    edgecolor='black', linewidth=2)
    ax3.set_title('Paso 3: Probabilidades (Softmax)', fontsize=12, fontweight='bold', pad=10)
    ax3.set_ylabel('Probabilidad', fontsize=10)
    ax3.set_xticks(range(3))
    ax3.set_xticklabels(clases, fontsize=10)
    ax3.set_ylim([0, 1.2])
    ax3.axhline(y=1.0, color='red', linestyle='--', linewidth=1, alpha=0.5)
    ax3.grid(axis='y', alpha=0.3)
    
    for i, (bar, val) in enumerate(zip(bars3, probabilidades)):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03, 
                f'{val:.1%}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Mostrar suma (posicionada arriba sin solaparse)
    suma_prob = np.sum(probabilidades)
    ax3.text(1, 1.1, f'Suma = {suma_prob:.4f}', 
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7, pad=0.5),
            fontsize=10, fontweight='bold', ha='center', va='center')
    
    # --- GRAFICA 4: Diagrama de flujo ---
    ax4 = fig.add_subplot(gs[1, :])
    ax4.set_xlim(0, 10)
    ax4.set_ylim(0, 3)
    ax4.axis('off')
    ax4.set_title('Proceso Completo de Softmax', fontsize=13, fontweight='bold', pad=15)
    
    # Ecuación paso a paso
    y_pos = 1.8
    ax4.text(1, y_pos, 'Logits\n(z)', ha='center', va='center', 
            fontsize=10, bbox=dict(boxstyle='round', facecolor='#FFB6C1', alpha=0.7))
    ax4.annotate('', xy=(1.8, y_pos), xytext=(1.4, y_pos),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    ax4.text(2.5, y_pos, 'z - max(z)', ha='center', va='center', 
            fontsize=9, bbox=dict(boxstyle='round', facecolor='#DDA0DD', alpha=0.7))
    ax4.annotate('', xy=(3.5, y_pos), xytext=(3.0, y_pos),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    ax4.text(4.5, y_pos, 'exp(z\')', ha='center', va='center', 
            fontsize=10, bbox=dict(boxstyle='round', facecolor='#87CEEB', alpha=0.7))
    ax4.annotate('', xy=(5.5, y_pos), xytext=(5.1, y_pos),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    ax4.text(6.8, y_pos, 'Dividir\npor Suma', ha='center', va='center', 
            fontsize=9, bbox=dict(boxstyle='round', facecolor='#FFD700', alpha=0.7))
    ax4.annotate('', xy=(8.0, y_pos), xytext=(7.5, y_pos),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    ax4.text(9, y_pos, 'Probabilidades\nSuma = 1', ha='center', va='center', 
            fontsize=10, bbox=dict(boxstyle='round', facecolor='#90EE90', alpha=0.7))
    
    # Notas (bien separadas verticalmente)
    ax4.text(2.5, 0.7, 'Prevencion de overflow', ha='center', fontsize=9, 
            style='italic', color='purple')
    ax4.text(6.8, 0.7, 'Normalizacion', ha='center', fontsize=9, 
            style='italic', color='darkblue')
    
    if temperatura != 1.0:
        ax4.text(5, 0.1, f'Temperatura = {temperatura:.2f}', 
                ha='center', fontsize=9, fontweight='bold', color='darkred',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.6))
    
    # --- GRAFICA 5: Comparación de temperaturas ---
    ax5 = fig.add_subplot(gs[2, :])
    temperaturas = [0.5, 1.0, 2.0, 5.0]
    x = np.arange(3)
    width = 0.18
    
    for i, T in enumerate(temperaturas):
        probs_temp = softmax_con_temp(logits, T)
        offset = (i - 1.5) * width
        ax5.bar(x + offset, probs_temp, width, label=f'T={T}', alpha=0.7)
    
    ax5.set_ylabel('Probabilidad', fontsize=10)
    ax5.set_title('Efecto de la Temperatura en Softmax', fontsize=12, fontweight='bold', pad=10)
    ax5.set_xticks(x)
    ax5.set_xticklabels(clases)
    ax5.legend(fontsize=9, title='Temperatura', title_fontsize=10, loc='upper left')
    ax5.grid(axis='y', alpha=0.3)
    ax5.set_ylim([0, 1.05])
    ax5.axhline(y=1/3, color='gray', linestyle=':', linewidth=1, alpha=0.5)
    
    # --- GRAFICA 6: Heatmap de transformación ---
    ax6 = fig.add_subplot(gs[3, 0])
    data_matrix = np.vstack([logits, exp_values, probabilidades]).T
    im = ax6.imshow(data_matrix, cmap='YlOrRd', aspect='auto')
    ax6.set_title('Transformacion por Paso', fontsize=11, fontweight='bold', pad=10)
    ax6.set_xticks([0, 1, 2])
    ax6.set_xticklabels(['Logits', 'Exp', 'Softmax'], fontsize=9)
    ax6.set_yticks([0, 1, 2])
    ax6.set_yticklabels(clases, fontsize=9)
    
    for i in range(3):
        for j in range(3):
            color = 'white' if data_matrix[i, j] > np.max(data_matrix)*0.6 else 'black'
            ax6.text(j, i, f'{data_matrix[i, j]:.2f}', 
                    ha='center', va='center', color=color, fontweight='bold', fontsize=9)
    
    plt.colorbar(im, ax=ax6, fraction=0.046, pad=0.04)
    
    # --- GRAFICA 7: Fórmula matemática ---
    ax7 = fig.add_subplot(gs[3, 1:])
    ax7.set_xlim(0, 10)
    ax7.set_ylim(0, 4.5)
    ax7.axis('off')
    
    # Título
    ax7.text(5, 4.0, 'Formula de Softmax', ha='center', fontsize=13, fontweight='bold')
    
    # Fórmula principal
    formula = r'$\text{Softmax}(z_i) = \frac{e^{z_i - \max(z)}}{\sum_{j=1}^{n} e^{z_j - \max(z)}}$'
    ax7.text(5, 3.0, formula, ha='center', fontsize=18, 
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    # Ejemplo numérico con el primer valor
    ax7.text(5, 2.0, 'Ejemplo para Gato:', ha='center', fontsize=11, fontweight='bold')
    ejemplo = f'Softmax({logits[0]:.2f}) = {exp_values[0]:.3f} / {suma_exp:.3f} = {probabilidades[0]:.4f}'
    ax7.text(5, 1.5, ejemplo, ha='center', fontsize=10, 
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.6))
    
    # Propiedades clave (bien espaciadas)
    ax7.text(5, 0.9, 'Salidas entre 0 y 1', ha='center', fontsize=10, 
            color='darkgreen', fontweight='bold')
    ax7.text(5, 0.5, 'La suma de todas las salidas = 1', ha='center', fontsize=10, 
            color='darkgreen', fontweight='bold')
    ax7.text(5, 0.1, 'Numericamente estable (no overflow)', ha='center', fontsize=10, 
            color='darkgreen', fontweight='bold')
    
    plt.show()
    
    # Información numérica
    print("=" * 80)
    print("TRANSFORMACION DE SOFTMAX PASO A PASO")
    print("=" * 80)
    print(f"\nPASO 1 - LOGITS (Valores de entrada):")
    for i, (clase, logit) in enumerate(zip(clases, logits)):
        print(f"   {clase:8s}: {logit:8.3f}")
    print(f"\n   -> Estos son valores sin restriccion (pueden ser negativos, > 1, etc.)")
    
    print(f"\nPASO 2 - PREVENCION DE OVERFLOW:")
    print(f"   Maximo de logits: {np.max(logits):.3f}")
    print(f"   Logits ajustados (z - max):")
    for i, (clase, val) in enumerate(zip(clases, logits_shifted)):
        print(f"      {clase:8s}: {val:8.3f}")
    print(f"\n   -> Restar el maximo no cambia el resultado final, solo evita exp(valores grandes)")
    
    print(f"\nPASO 3 - EXPONENCIALES:")
    for i, (clase, exp_val) in enumerate(zip(clases, exp_values)):
        print(f"   exp({logits_shifted[i]:6.3f}) = {exp_val:8.3f}  [{clase}]")
    print(f"\n   Suma de exponenciales: {suma_exp:.3f}")
    print(f"   -> Todos los valores ahora son positivos")
    
    print(f"\nPASO 4 - NORMALIZACION (Dividir por la suma):")
    for clase, exp_val, prob in zip(clases, exp_values, probabilidades):
        barra = '█' * int(prob * 50)
        print(f"   {clase:8s}: {exp_val:6.3f} / {suma_exp:.3f} = {prob:.4f} ({prob:6.2%}) {barra}")
    
    print(f"\n   Suma total: {np.sum(probabilidades):.6f} (aprox. 1.0)")
    print(f"   -> Ahora los valores estan entre 0 y 1 y suman 1 (son probabilidades)")
    
    if temperatura != 1.0:
        print(f"\nTEMPERATURA: {temperatura}")
        if temperatura < 1:
            print("   -> T < 1: Las diferencias entre clases se amplifican")
            print("   -> La clase ganadora tiene probabilidad mas alta")
        elif temperatura > 1:
            print("   -> T > 1: Las diferencias se suavizan")
            print("   -> Las probabilidades son mas parecidas entre si")
    
    print(f"\nINTERPRETACION:")
    clase_max = clases[np.argmax(probabilidades)]
    prob_max = np.max(probabilidades)
    print(f"   La clase con mayor probabilidad es: {clase_max} con {prob_max:.1%}")
    print(f"   Esto significa que el modelo 'cree' que la entrada es mas probable que sea {clase_max}")
    
    print("=" * 80)

# Crear controles
slider_temp = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='Temperatura:',
    continuous_update=False,
    style={'description_width': '120px'}
)

slider_magnitud = widgets.FloatSlider(
    value=1.0, min=0.5, max=3.0, step=0.1,
    description='Magnitud Logits:',
    continuous_update=False,
    style={'description_width': '120px'}
)

# Interfaz interactiva
print("VISUALIZACION INTERACTIVA DE SOFTMAX")
print("=" * 80)
print("Ajusta los parametros para explorar como Softmax transforma logits en probabilidades:\n")
print("* Temperatura: Controla que tan 'suave' o 'picuda' es la distribucion")
print("* Magnitud Logits: Cambia la escala de los valores de entrada\n")

widgets.interact(visualizar_softmax,
                temperatura=slider_temp,
                magnitud_logits=slider_magnitud)

VISUALIZACION INTERACTIVA DE SOFTMAX
Ajusta los parametros para explorar como Softmax transforma logits en probabilidades:

* Temperatura: Controla que tan 'suave' o 'picuda' es la distribucion
* Magnitud Logits: Cambia la escala de los valores de entrada



interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Temperatura:', max=5.0, min…

<function __main__.visualizar_softmax(temperatura, magnitud_logits)>

## Calculo de perdidas



## Función de Error (Loss) y Entropía Cruzada Categórica

Hasta ahora, nuestra red neuronal ya puede **calcular salidas y probabilidades** gracias a Softmax.  

Pero... ¿cómo sabemos si esas salidas están "bien" o "mal"?  

Ahí entra la **función de error (o pérdida)**.


### ¿Por qué medimos el error y no la precisión?

Podríamos intentar medir la **precisión** (porcentaje de aciertos), pero esa medida es **demasiado gruesa**:  
solo nos dice si una predicción es correcta o no, sin dar información sobre *cuánto* se equivocó la red.

Por ejemplo:  
- Si la red predice `[0.6, 0.4]` para una clase real `[1, 0]`, no está del todo mal.  
- Pero si predice `[0.51, 0.49]`, está muy insegura, aunque la precisión sea igual (acertó).  

Para **aprender correctamente**, necesitamos una medida continua que capture esa diferencia.  
Esa medida es la **función de pérdida**, que le da al modelo una forma cuantitativa de saber *cuánto se está equivocando*.


### ¿Qué es la Entropía Cruzada?

La **entropía cruzada categórica (Categorical Cross-Entropy)** mide la diferencia entre la **distribución real** de las clases y la **distribución predicha** por el modelo.

Matemáticamente:

$$
L = -\sum_i y_i \cdot \log(\hat{y}_i)
$$

Donde:
- $y_i$ es la etiqueta real (1 para la clase correcta, 0 para las demás).
- $\hat{y}_i$ es la probabilidad predicha por la red para esa clase.

Esta función penaliza más cuando el modelo da una **baja probabilidad a la clase correcta**, y menos cuando se acerca a 1.


### Intuición

Imagina que tu red tiene que decidir entre tres clases, y la respuesta correcta es la clase 1.  

| Predicción | Interpretación | Pérdida |
|-------------|----------------|---------|
| [0.9, 0.05, 0.05] | Confía en la clase correcta | Baja |
| [0.33, 0.33, 0.34] | Duda mucho | Media |
| [0.05, 0.9, 0.05] | Segura pero incorrecta | Alta |

Así, la red aprende a **aumentar la probabilidad de la clase correcta** y disminuir las otras.



### En resumen

- El **error (loss)** guía el aprendizaje de la red.  
- La **precisión** sirve para evaluar resultados, pero no para entrenar.  
- La **entropía cruzada categórica** es la más usada cuando hay múltiples clases de salida.  
- Se combina naturalmente con **Softmax**, porque Softmax produce una distribución de probabilidades.



## Ejemplo en Python

A continuación implementamos la **entropía cruzada categórica** y la usamos para calcular el error entre las salidas de Softmax y las clases reales.

In [19]:
import numpy as np

# ======== Clases previas ========

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        # Pesos: matriz (n_inputs × n_neurons)
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        # Biases: vector fila (1 × n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        # Producto punto + bias
        self.output = np.dot(inputs, self.weights) + self.biases

class Activation_ReLU:
    def forward(self, inputs):
        # ReLU: reemplaza valores negativos por 0
        self.output = np.maximum(0, inputs)

class Activation_Softmax:
    def forward(self, inputs):
        # Estabilización numérica restando el máximo
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        # Normalización por el total de exponentes
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

# ======== Clase de función de pérdida ========

class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        # Evitar log(0) o valores extremos
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Si las etiquetas son one-hot (matriz)
        if len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        # Si las etiquetas son índices (vector)
        elif len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(len(y_pred)), y_true]

        # Pérdida individual por muestra
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

# ======== Ejemplo práctico ========

# Entradas: 3 muestras con 4 características
X = np.array([
    [1.0, 2.0, 3.0, 2.5],
    [2.0, 5.0, -1.0, 2.0],
    [-1.5, 2.7, 3.3, -0.8]
])

# Etiquetas reales: clases correctas (0, 1, 1)
y_true = np.array([0, 1, 1])   # <--- convertimos a array

# Definición de la red
np.random.seed(0)
layer1 = Layer_Dense(4, 5)
activation1 = Activation_ReLU()
layer2 = Layer_Dense(5, 3)
activation2 = Activation_Softmax()
loss_function = Loss_CategoricalCrossentropy()

# Forward pass
layer1.forward(X)
activation1.forward(layer1.output)
layer2.forward(activation1.output)
activation2.forward(layer2.output)

# Cálculo de la pérdida
loss = loss_function.forward(activation2.output, y_true)
average_loss = np.mean(loss)

print("Predicciones (probabilidades):")
print(activation2.output)
print("\nPérdidas individuales:")
print(loss)
print(f"\nPérdida promedio total: {average_loss:.4f}")



Predicciones (probabilidades):
[[0.30326886 0.40108669 0.29564444]
 [0.32834771 0.36829707 0.30335522]
 [0.31750561 0.37292873 0.30956567]]

Pérdidas individuales:
[1.19313553 0.99886542 0.98636796]

Pérdida promedio total: 1.0595


Ahora un ejemplo interactivo de la cross-entropía:

In [20]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

# ======== Clases de la red ========
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        if len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        elif len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(len(y_pred)), y_true]
        
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

def visualizar_loss(mult_pesos1, mult_pesos2, seed):
    """Visualiza el cálculo de loss y cross-entropy"""
    
    # Datos de entrada
    X = np.array([
        [1.0, 2.0, 3.0, 2.5],
        [2.0, 5.0, -1.0, 2.0],
        [-1.5, 2.7, 3.3, -0.8]
    ])
    
    # Etiquetas verdaderas
    y_true = np.array([0, 1, 1])
    nombres_clases = ['Clase 0', 'Clase 1', 'Clase 2']
    n_muestras = len(X)
    n_clases = 3
    
    # Configurar red
    np.random.seed(seed)
    layer1 = Layer_Dense(4, 5)
    layer1.weights *= mult_pesos1
    activation1 = Activation_ReLU()
    layer2 = Layer_Dense(5, 3)
    layer2.weights *= mult_pesos2
    activation2 = Activation_Softmax()
    loss_function = Loss_CategoricalCrossentropy()
    
    # Forward pass
    layer1.forward(X)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    activation2.forward(layer2.output)
    
    # Calcular pérdida
    loss = loss_function.forward(activation2.output, y_true)
    average_loss = np.mean(loss)
    
    # Calcular precisión
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == y_true)
    
    # Cerrar figuras previas
    plt.close('all')
    
    # Crear visualización
    fig = plt.figure(figsize=(16, 13))
    gs = fig.add_gridspec(4, 3, hspace=0.5, wspace=0.35, top=0.96, bottom=0.04)
    
    # --- GRÁFICA 1: Heatmap de probabilidades predichas ---
    ax1 = fig.add_subplot(gs[0, :2])
    im1 = ax1.imshow(activation2.output, cmap='YlGnBu', aspect='auto', vmin=0, vmax=1)
    ax1.set_title('Probabilidades Predichas por la Red (Salida de Softmax)', 
                  fontsize=12, fontweight='bold', pad=20)
    ax1.set_xlabel('Clase', fontsize=10)
    ax1.set_ylabel('Muestra', fontsize=10)
    ax1.set_xticks(range(n_clases))
    ax1.set_yticks(range(n_muestras))
    ax1.set_xticklabels(nombres_clases, fontsize=9)
    ax1.set_yticklabels([f'Muestra {i}' for i in range(n_muestras)], fontsize=9)
    
    # Añadir valores y marcar clases correctas
    for i in range(n_muestras):
        for j in range(n_clases):
            value = activation2.output[i, j]
            color = 'white' if value > 0.5 else 'black'
            
            # Marcar la clase correcta con un borde
            if j == y_true[i]:
                rect = plt.Rectangle((j-0.45, i-0.45), 0.9, 0.9, 
                                    fill=False, edgecolor='red', linewidth=3)
                ax1.add_patch(rect)
                ax1.text(j, i, f'{value:.3f}\n(REAL)', ha='center', va='center', 
                        color=color, fontsize=8, fontweight='bold')
            else:
                ax1.text(j, i, f'{value:.3f}', ha='center', va='center', 
                        color=color, fontsize=8)
    
    plt.colorbar(im1, ax=ax1, label='Probabilidad', fraction=0.046, pad=0.04)
    
    # --- GRÁFICA 2: Curva teórica de Cross-Entropy ---
    ax2 = fig.add_subplot(gs[0, 2])
    probs = np.linspace(0.01, 1, 1000)
    ce_values = -np.log(probs)
    
    ax2.plot(probs, ce_values, linewidth=2.5, color='#E74C3C')
    ax2.fill_between(probs, ce_values, alpha=0.3, color='#E74C3C')
    ax2.set_xlabel('Probabilidad de la clase correcta', fontsize=9)
    ax2.set_ylabel('Loss (Cross-Entropy)', fontsize=9)
    ax2.set_title('Curva de Penalizacion\n-log(p)', fontsize=11, fontweight='bold', pad=15)
    ax2.grid(alpha=0.3)
    ax2.set_ylim([0, 5])
    
    # Marcar puntos de referencia
    ref_probs = [0.1, 0.5, 0.9]
    for p in ref_probs:
        loss_val = -np.log(p)
        ax2.plot(p, loss_val, 'o', markersize=8, color='darkred')
        ax2.text(p, loss_val + 0.3, f'p={p}\nL={loss_val:.2f}', 
                ha='center', fontsize=7, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
    
    # --- GRÁFICA 3: Comparación por muestra ---
    ax3 = fig.add_subplot(gs[1, :])
    x = np.arange(n_muestras)
    width = 0.25
    
    # Barras para cada clase
    colores_clases = ['#3498DB', '#2ECC71', '#F39C12']
    for j in range(n_clases):
        offset = (j - 1) * width
        bars = ax3.bar(x + offset, activation2.output[:, j], width, 
                      label=nombres_clases[j], color=colores_clases[j], alpha=0.8)
        
        # Resaltar la clase correcta para cada muestra
        for i in range(n_muestras):
            if j == y_true[i]:
                bars[i].set_edgecolor('red')
                bars[i].set_linewidth(3)
    
    ax3.set_xlabel('Muestra', fontsize=10)
    ax3.set_ylabel('Probabilidad', fontsize=10)
    ax3.set_title('Distribucion de Probabilidades por Muestra (Borde rojo = Clase correcta)', 
                  fontsize=12, fontweight='bold', pad=20)
    ax3.set_xticks(x)
    ax3.set_xticklabels([f'M{i}\n(Real: {y_true[i]})' for i in range(n_muestras)], fontsize=9)
    ax3.legend(loc='upper right', fontsize=9)
    ax3.grid(axis='y', alpha=0.3)
    ax3.set_ylim([0, 1.15])
    
    # Línea en 1.0 para mostrar que suman 1
    ax3.axhline(y=1.0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    
    # --- GRÁFICA 4: Pérdida individual por muestra ---
    ax4 = fig.add_subplot(gs[2, 0])
    colores_loss = ['#27AE60' if l < 0.5 else '#E67E22' if l < 1.5 else '#E74C3C' for l in loss]
    bars4 = ax4.bar(range(n_muestras), loss, color=colores_loss, alpha=0.8, edgecolor='black')
    ax4.set_xlabel('Muestra', fontsize=10)
    ax4.set_ylabel('Loss Individual', fontsize=10)
    ax4.set_title('Perdida por Muestra', fontsize=11, fontweight='bold', pad=15)
    ax4.set_xticks(range(n_muestras))
    ax4.set_xticklabels([f'M{i}' for i in range(n_muestras)], fontsize=9)
    ax4.grid(axis='y', alpha=0.3)
    
    # Calcular límites con margen
    max_loss = np.max(loss)
    ax4.set_ylim([0, max_loss * 1.25])
    
    # Añadir valores sobre las barras
    for i, (bar, l) in enumerate(zip(bars4, loss)):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + max_loss * 0.03,
                f'{l:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # Línea del promedio
    ax4.axhline(y=average_loss, color='red', linestyle='--', linewidth=2, 
               label=f'Promedio: {average_loss:.3f}')
    ax4.legend(fontsize=8)
    
    # --- GRÁFICA 5: Confianza en la clase correcta ---
    ax5 = fig.add_subplot(gs[2, 1])
    confidencias = activation2.output[range(n_muestras), y_true]
    colores_conf = ['#27AE60' if c > 0.7 else '#E67E22' if c > 0.4 else '#E74C3C' for c in confidencias]
    bars5 = ax5.bar(range(n_muestras), confidencias, color=colores_conf, alpha=0.8, edgecolor='black')
    ax5.set_xlabel('Muestra', fontsize=10)
    ax5.set_ylabel('Probabilidad', fontsize=10)
    ax5.set_title('Confianza en la Clase Correcta', fontsize=11, fontweight='bold', pad=15)
    ax5.set_xticks(range(n_muestras))
    ax5.set_xticklabels([f'M{i}' for i in range(n_muestras)], fontsize=9)
    ax5.set_ylim([0, 1.15])
    ax5.grid(axis='y', alpha=0.3)
    
    # Añadir valores
    for i, (bar, c) in enumerate(zip(bars5, confidencias)):
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height + 0.03,
                f'{c:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # Zonas de referencia
    ax5.axhspan(0.7, 1.0, alpha=0.1, color='green', label='Confiado')
    ax5.axhspan(0.4, 0.7, alpha=0.1, color='orange', label='Dudoso')
    ax5.axhspan(0, 0.4, alpha=0.1, color='red', label='Inseguro')
    ax5.legend(fontsize=7, loc='lower right')
    
    # --- GRÁFICA 6: Predicciones vs Verdad ---
    ax6 = fig.add_subplot(gs[2, 2])
    ax6.axis('off')
    ax6.set_title('Predicciones vs Realidad', fontsize=11, fontweight='bold', pad=15)
    ax6.set_xlim([0, 1])
    ax6.set_ylim([0, 1])
    
    y_text = 0.95
    for i in range(n_muestras):
        pred_class = predictions[i]
        true_class = y_true[i]
        pred_prob = activation2.output[i, pred_class]
        true_prob = activation2.output[i, true_class]
        
        is_correct = pred_class == true_class
        color = 'green' if is_correct else 'red'
        symbol = 'OK' if is_correct else 'ERROR'
        
        text = f'Muestra {i}:\n'
        text += f'  Pred: Clase {pred_class} (p={pred_prob:.3f})\n'
        text += f'  Real: Clase {true_class} (p={true_prob:.3f})\n'
        text += f'  [{symbol}] Loss: {loss[i]:.3f}'
        
        ax6.text(0.05, y_text, text, fontsize=8, family='monospace',
                bbox=dict(boxstyle='round', facecolor=color, alpha=0.2),
                verticalalignment='top', transform=ax6.transAxes)
        y_text -= 0.30
    
    # --- GRÁFICA 7: Métricas generales ---
    ax7 = fig.add_subplot(gs[3, :])
    ax7.axis('off')
    ax7.set_xlim([0, 1])
    ax7.set_ylim([0, 1])
    
    # Cuadro de métricas
    metricas_text = f'''METRICAS DE LA RED
══════════════════════════════════════════════════════════════
LOSS (Perdida):
  - Promedio:         {average_loss:.4f}
  - Minimo (mejor):   {np.min(loss):.4f}  (Muestra {np.argmin(loss)})
  - Maximo (peor):    {np.max(loss):.4f}  (Muestra {np.argmax(loss)})

PRECISION (Accuracy):
  - Aciertos:         {int(accuracy * n_muestras)}/{n_muestras}
  - Porcentaje:       {accuracy * 100:.1f}%

INTERPRETACION DEL LOSS:
  - BAJO  (<0.5):  Red muy confiada en la clase correcta
  - MEDIO (0.5-1.5): Red con dudas, pero razonable
  - ALTO  (>1.5):  Red equivocada o muy insegura

DIFERENCIA CLAVE:
  - PRECISION: Solo cuenta si acierta o no (binario: 0 o 1)
  - LOSS: Mide CUAN segura esta (continuo, diferenciable)
  
  Usamos LOSS para entrenar (permite gradientes)
  y PRECISION para evaluar resultados finales.'''
    
    ax7.text(0.5, 0.5, metricas_text, fontsize=9, family='monospace',
            ha='center', va='center', transform=ax7.transAxes,
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))
    
    plt.show()
    
    # Información en consola
    print("=" * 80)
    print("ANALISIS DETALLADO DE LOSS Y CROSS-ENTROPY")
    print("=" * 80)
    print(f"\nPROBABILIDADES PREDICHAS:")
    print(activation2.output)
    print(f"\nCLASES VERDADERAS: {y_true}")
    print(f"CLASES PREDICHAS:  {predictions}")
    print(f"\nCONFIANZA EN CLASE CORRECTA:")
    for i in range(n_muestras):
        print(f"  Muestra {i}: {confidencias[i]:.4f} -> Loss: {loss[i]:.4f}")
    print(f"\nLOSS PROMEDIO:     {average_loss:.4f}")
    print(f"PRECISION:         {accuracy*100:.1f}%")
    print("=" * 80)

# Crear sliders
slider_pesos1 = widgets.FloatSlider(
    value=1.0, min=0.1, max=3.0, step=0.1,
    description='Mult. Pesos L1:',
    continuous_update=False,
    style={'description_width': '120px'}
)

slider_pesos2 = widgets.FloatSlider(
    value=1.0, min=0.1, max=3.0, step=0.1,
    description='Mult. Pesos L2:',
    continuous_update=False,
    style={'description_width': '120px'}
)

slider_seed = widgets.IntSlider(
    value=0, min=0, max=10, step=1,
    description='Semilla:',
    continuous_update=False,
    style={'description_width': '120px'}
)

# Crear interfaz interactiva
print("Ajusta los parametros para ver como cambia el loss segun las predicciones\n")
widgets.interact(visualizar_loss,
                mult_pesos1=slider_pesos1,
                mult_pesos2=slider_pesos2,
                seed=slider_seed)

Ajusta los parametros para ver como cambia el loss segun las predicciones



interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Mult. Pesos L1:', max=3.0, …

<function __main__.visualizar_loss(mult_pesos1, mult_pesos2, seed)>

## Optimizacion

## Optimización y ajuste de parámetros

Hasta este punto, hemos visto cómo una red neuronal calcula sus salidas a partir de las entradas, los **pesos** y los **sesgos**.

Sin embargo, el verdadero poder de las redes neuronales radica en su capacidad de **aprender automáticamente** los valores de esos parámetros a través de un proceso conocido como **optimización**.


### ¿Por qué necesitamos optimizadores?

En cada iteración del entrenamiento, la red produce una predicción y la compara con el valor real (la etiqueta verdadera).

A partir de esta comparación se calcula una **función de pérdida**, que mide qué tan lejos está la predicción de la verdad.

El objetivo del optimizador es **ajustar los pesos y sesgos de manera que la pérdida disminuya**.

Dicho de forma intuitiva, el optimizador es el **mecanismo que entrena el modelo**: decide, con base en el error actual, **cómo y en qué dirección** deben modificarse los pesos para mejorar la precisión de la red.


### ¿Por qué es un concepto complicado?

El problema de optimización de una red neuronal no es trivial:

- El espacio de parámetros (todos los pesos y sesgos) puede ser **enorme**.
- La superficie de error puede ser **altamente no lineal**, con muchos mínimos locales.
- Ajustar un parámetro puede afectar indirectamente a muchos otros.

Por eso, los optimizadores modernos (como *SGD*, *Momentum*, *RMSProp* o *Adam*) implementan estrategias matemáticas sofisticadas para **ajustar cada peso de manera inteligente**, sin que nosotros tengamos que resolver manualmente el problema.


### Intuición del proceso de ajuste

Podemos imaginar el proceso de **optimización** como caminar por una superficie montañosa tratando de alcanzar el punto más bajo, que representa la menor pérdida posible. En cada paso de este recorrido, el modelo analiza la forma de esa superficie y decide cómo ajustar sus parámetros para acercarse al mínimo.

En términos generales, el proceso sigue tres pasos:

1. Se calcula la **pendiente** de la superficie, es decir, el **gradiente** de la función de pérdida respecto a cada peso.  
2. Se actualizan los pesos moviéndose en la dirección contraria a esa pendiente, ya que es la que reduce la pérdida.  
3. Se repite el procedimiento hasta que los cambios sean mínimos y la pérdida se estabilice en un valor suficientemente bajo.

Matemáticamente, la actualización de un peso $w$ se expresa como:

$$
w_{\text{nuevo}} = w_{\text{viejo}} - \eta \frac{\partial L}{\partial w}
$$

donde:  
- $\eta$ es la **tasa de aprendizaje** (*learning rate*), que controla el tamaño de cada paso.  
- $\frac{\partial L}{\partial w}$ representa el **gradiente** de la pérdida con respecto al peso.

Durante este proceso —también conocido como **entrenamiento de la red neuronal**— se distinguen dos fases principales: el **forward pass** y el **backward pass**. En el *forward pass*, los datos de entrada atraviesan la red capa por capa, aplicando pesos, sesgos y funciones de activación hasta producir una salida o predicción. Luego, en el *backward pass*, se calcula el error entre la predicción y el valor real, y se utiliza el algoritmo de **backpropagation** para propagar ese error hacia atrás por la red. Gracias a este mecanismo, los gradientes obtenidos permiten actualizar los pesos de forma eficiente, haciendo que el modelo aprenda patrones cada vez más precisos y cierre así el ciclo completo de aprendizaje supervisado.

### Cómo simplificamos esto en la práctica

Implementar estos cálculos desde cero puede ser tedioso y propenso a errores.

Por eso, librerías como **NumPy**, **TensorFlow** o **PyTorch** nos permiten centrarnos en la arquitectura y la lógica, mientras ellas se encargan de calcular los gradientes y actualizar los parámetros automáticamente mediante los optimizadores.

De hecho, en frameworks modernos basta con definir el optimizador y la función de pérdida; el resto (diferenciación automática, propagación del error y actualización de parámetros) se maneja internamente.

### En resumen

- Los **optimizadores** son el corazón del proceso de aprendizaje.
- Permiten ajustar los pesos y sesgos reduciendo progresivamente la pérdida.
- Su papel es fundamental: **sin un optimizador, el modelo nunca aprendería**.


In [21]:
import numpy as np

# Datos de entrada (4 muestras, 3 características cada una)
X = np.array([[1.0, 2.0, 3.0],
              [2.0, 3.0, 4.0],
              [3.0, 4.0, 5.0],
              [4.0, 5.0, 6.0]])

# Etiquetas verdaderas (clasificación binaria: 0 o 1)
y_true = np.array([[0],
                   [0],
                   [1],
                   [1]])

# Fijar semilla para reproducibilidad
np.random.seed(0)

# Capa densa
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        # Pesos aleatorios pequeños
        self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
        # Biases inicializados en cero
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs):
        # Guardar las entradas para usar en backprop
        self.inputs = inputs
        # Producto punto entre entradas y pesos + biases
        self.output = np.dot(inputs, self.weights) + self.biases

# Función de activación Sigmoid (para clasificación binaria)
class Activation_Sigmoid:
    def forward(self, inputs):
        self.output = 1 / (1 + np.exp(-inputs))

# Función de pérdida: Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Añadir pequeño epsilon para evitar log(0)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calcular pérdida
        loss = -np.mean(y_true * np.log(y_pred_clipped) + 
                        (1 - y_true) * np.log(1 - y_pred_clipped))
        return loss

# Optimizador: Stochastic Gradient Descent (SGD)
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0):
        self.learning_rate = learning_rate
    
    def update_params(self, layer, dweights, dbiases):
        # Actualizar pesos: w_nuevo = w_viejo - learning_rate * gradiente
        layer.weights -= self.learning_rate * dweights
        layer.biases -= self.learning_rate * dbiases

# Crear red neuronal simple
# 3 entradas -> 1 neurona de salida (clasificación binaria)
layer1 = Layer_Dense(3, 1)
activation1 = Activation_Sigmoid()
loss_function = Loss_BinaryCrossEntropy()
optimizer = Optimizer_SGD(learning_rate=0.5)

print("=" * 60)
print("ENTRENAMIENTO CON OPTIMIZADOR SGD")
print("=" * 60)
print(f"\nPesos iniciales:\n{layer1.weights}")
print(f"\nBiases iniciales:\n{layer1.biases}")

# Entrenar por varias épocas
n_epochs = 5
for epoch in range(n_epochs):
    # Forward pass
    layer1.forward(X)
    activation1.forward(layer1.output)
    y_pred = activation1.output
    
    # Calcular pérdida
    loss = loss_function.forward(y_pred, y_true)
    
    # Calcular gradientes (simplificado para este ejemplo)
    # En la práctica, esto se hace con backpropagation
    # Gradiente de la pérdida respecto a las predicciones
    dloss = y_pred - y_true
    
    # Gradientes respecto a pesos y biases
    dweights = np.dot(layer1.inputs.T, dloss) / len(X)
    dbiases = np.mean(dloss, axis=0, keepdims=True)
    
    # Actualizar parámetros usando el optimizador
    optimizer.update_params(layer1, dweights, dbiases)
    
    # Mostrar progreso
    print(f"\nEpoca {epoch + 1}:")
    print(f"  Perdida: {loss:.4f}")
    print(f"  Predicciones: {y_pred.T.flatten()}")

# Resultados finales
print("\n" + "=" * 60)
print("RESULTADOS FINALES")
print("=" * 60)
print(f"\nPesos finales:\n{layer1.weights}")
print(f"\nBiases finales:\n{layer1.biases}")

# Forward pass final para ver predicciones
layer1.forward(X)
activation1.forward(layer1.output)
y_pred_final = activation1.output
loss_final = loss_function.forward(y_pred_final, y_true)

print(f"\nPredicciones finales:")
for i, (pred, true) in enumerate(zip(y_pred_final, y_true)):
    clase_pred = "Clase 1" if pred[0] > 0.5 else "Clase 0"
    clase_true = "Clase 1" if true[0] == 1 else "Clase 0"
    print(f"  Muestra {i}: pred={pred[0]:.4f} ({clase_pred}) | "
          f"real={clase_true} | {'✓' if (pred[0] > 0.5) == true[0] else '✗'}")

print(f"\nPerdida final: {loss_final:.4f}")
print("\nNota: Los pesos se han ajustado automaticamente para reducir la perdida")
print("=" * 60)

ENTRENAMIENTO CON OPTIMIZADOR SGD

Pesos iniciales:
[[0.17640523]
 [0.04001572]
 [0.0978738 ]]

Biases iniciales:
[[0.]]

Epoca 1:
  Perdida: 0.6731
  Predicciones: [0.63414906 0.70356928 0.76470455 0.81651949]

Epoca 2:
  Perdida: 1.0311
  Predicciones: [0.28193946 0.23312932 0.1905272  0.15414535]

Epoca 3:
  Perdida: 2.5950
  Predicciones: [0.98441377 0.99800727 0.99974826 0.99996825]

Epoca 4:
  Perdida: 0.5950
  Predicciones: [0.42094248 0.47279223 0.52523559 0.57712813]

Epoca 5:
  Perdida: 0.8526
  Predicciones: [0.7305518  0.86585825 0.93890156 0.97339328]

RESULTADOS FINALES

Pesos finales:
[[ 0.3088204 ]
 [-0.23577219]
 [-0.58611718]]

Biases finales:
[[-0.40820307]]

Predicciones finales:
  Muestra 0: pred=0.0887 (Clase 0) | real=Clase 0 | ✓
  Muestra 1: pred=0.0551 (Clase 0) | real=Clase 0 | ✓
  Muestra 2: pred=0.0337 (Clase 0) | real=Clase 1 | ✗
  Muestra 3: pred=0.0205 (Clase 0) | real=Clase 1 | ✗

Perdida final: 1.8571

Nota: Los pesos se han ajustado automaticamente par

## Implementacion en Tensorflow

A lo largo de los capítulos anteriores, se han construido **desde cero** los componentes fundamentales de una red neuronal:

- Capas densas con pesos y sesgos
- Funciones de activación (ReLU, Softmax)
- Procesamiento por batches
- Funciones de pérdida
- Optimizadores

Implementar estos elementos manualmente ha permitido entender **qué está pasando realmente** bajo el capó de una red neuronal.

Sin embargo, en la práctica, construir redes desde cero para cada proyecto sería **ineficiente y propenso a errores**.


### El rol de las librerías modernas

Aquí es donde entran librerías como **TensorFlow/Keras**, **PyTorch** y otras:

- **Simplifican el código**: Lo que nos tomó decenas de líneas ahora son unas pocas.
- **Optimizan el rendimiento**: Usan implementaciones altamente eficientes en C/C++ y aprovechan GPUs.
- **Manejan la complejidad**: Calculan gradientes automáticamente (diferenciación automática).
- **Reducen errores**: Código probado y mantenido por miles de desarrolladores.

### Comparación: Implementación manual vs TensorFlow

Veamos cómo todo lo que se ha construido se traduce a TensorFlow/Keras de forma extremadamente concisa.

## Ejemplo en Python

A continuación, se implementa **exactamente la misma arquitectura** que se ha venido trabajando, pero usando TensorFlow/Keras:

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("=" * 70)
print("RED NEURONAL CON TENSORFLOW/KERAS")
print("=" * 70)

# ============================================================================
# DATOS DE ENTRADA (los mismos que hemos usado)
# ============================================================================
X = np.array([[1.0, 2.0, 3.0, 2.5],
              [2.0, 5.0, -1.0, 2.0],
              [-1.5, 2.7, 3.3, -0.8]])

# Etiquetas para clasificación (3 clases)
y_true = np.array([[1, 0, 0],  # Clase 0
                   [0, 1, 0],  # Clase 1
                   [0, 0, 1]]) # Clase 2

print("\nDatos de entrada:")
print(f"X shape: {X.shape} (3 muestras, 4 caracteristicas)")
print(f"y shape: {y_true.shape} (3 muestras, 3 clases)\n")

# ============================================================================
# CONSTRUCCIÓN DEL MODELO
# ============================================================================
# Fijar semilla para reproducibilidad
tf.random.set_seed(0)
np.random.seed(0)

# Crear modelo secuencial (capa tras capa)
modelo = keras.Sequential([
    # Capa oculta: 4 entradas -> 5 neuronas con ReLU
    layers.Dense(5, activation='relu', input_shape=(4,), name='capa_oculta'),
    
    # Capa de salida: 5 entradas -> 3 neuronas con Softmax
    layers.Dense(3, activation='softmax', name='capa_salida')
])

print("ARQUITECTURA DEL MODELO:")
print("-" * 70)
modelo.summary()
print("-" * 70)

# ============================================================================
# COMPILACIÓN DEL MODELO
# ============================================================================
# Aquí especificamos:
# - Optimizador: SGD (Stochastic Gradient Descent)
# - Función de pérdida: Categorical Crossentropy (para clasificación multiclase)
# - Métricas: Accuracy (precisión)

modelo.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nMODELO COMPILADO:")
print("  - Optimizador: SGD (learning_rate=0.5)")
print("  - Funcion de perdida: Categorical Crossentropy")
print("  - Metricas: Accuracy")

# ============================================================================
# PREDICCIÓN ANTES DEL ENTRENAMIENTO
# ============================================================================
print("\n" + "=" * 70)
print("PREDICCIONES ANTES DEL ENTRENAMIENTO (pesos aleatorios)")
print("=" * 70)

y_pred_inicial = modelo.predict(X, verbose=0)
print("\nPredicciones (probabilidades):")
for i, pred in enumerate(y_pred_inicial):
    clase_pred = np.argmax(pred)
    clase_true = np.argmax(y_true[i])
    print(f"  Muestra {i}: {pred} -> Clase {clase_pred} | "
          f"Real: Clase {clase_true} | {'✓' if clase_pred == clase_true else '✗'}")

# Calcular pérdida inicial
loss_inicial = modelo.evaluate(X, y_true, verbose=0)[0]
print(f"\nPerdida inicial: {loss_inicial:.4f}")

# ============================================================================
# ENTRENAMIENTO
# ============================================================================
print("\n" + "=" * 70)
print("ENTRENAMIENTO")
print("=" * 70)

# Entrenar el modelo
# epochs: número de veces que se pasa por todos los datos
# verbose: controla cuánta información se imprime
history = modelo.fit(
    X, y_true,
    epochs=100,
    verbose=0  # Silencioso para no llenar la salida
)

print(f"\nEntrenamiento completado: {len(history.history['loss'])} epocas")
print(f"Perdida final: {history.history['loss'][-1]:.4f}")
print(f"Accuracy final: {history.history['accuracy'][-1]:.4f}")

# Mostrar evolución cada 20 épocas
print("\nEvolucion de la perdida:")
for i in [0, 19, 39, 59, 79, 99]:
    if i < len(history.history['loss']):
        print(f"  Epoca {i+1:3d}: Perdida = {history.history['loss'][i]:.4f}")

# ============================================================================
# PREDICCIÓN DESPUÉS DEL ENTRENAMIENTO
# ============================================================================
print("\n" + "=" * 70)
print("PREDICCIONES DESPUES DEL ENTRENAMIENTO")
print("=" * 70)

y_pred_final = modelo.predict(X, verbose=0)
print("\nPredicciones (probabilidades):")
for i, pred in enumerate(y_pred_final):
    clase_pred = np.argmax(pred)
    clase_true = np.argmax(y_true[i])
    print(f"  Muestra {i}: {pred} -> Clase {clase_pred} | "
          f"Real: Clase {clase_true} | {'✓' if clase_pred == clase_true else '✗'}")

# Evaluar modelo
loss_final, accuracy_final = modelo.evaluate(X, y_true, verbose=0)
print(f"\nPerdida final: {loss_final:.4f}")
print(f"Accuracy final: {accuracy_final:.4f}")

# ============================================================================
# INSPECCIÓN DE PESOS
# ============================================================================
print("\n" + "=" * 70)
print("INSPECCION DE PESOS APRENDIDOS")
print("=" * 70)

for i, layer in enumerate(modelo.layers):
    weights, biases = layer.get_weights()
    print(f"\n{layer.name}:")
    print(f"  Pesos shape: {weights.shape}")
    print(f"  Biases shape: {biases.shape}")
    print(f"  Pesos (primeras 3 filas):\n{weights[:3]}")
    print(f"  Biases: {biases}")

# ============================================================================
# COMPARACIÓN CON IMPLEMENTACIÓN MANUAL
# ============================================================================
print("\n" + "=" * 70)
print("COMPARACION: MANUAL vs TENSORFLOW")
print("=" * 70)

comparacion = """
┌─────────────────────────┬──────────────────────┬─────────────────────────┐
│ Componente              │ Implementacion Manual│ TensorFlow/Keras        │
├─────────────────────────┼──────────────────────┼─────────────────────────┤
│ Capa Densa              │ Clase Layer_Dense    │ layers.Dense()          │
│ Activación ReLU         │ Clase Activation_ReLU│ activation='relu'       │
│ Activación Softmax      │ Clase Activation_...│ activation='softmax'    │
│ Función de pérdida      │ Clase Loss_...       │ loss='categorical_...'  │
│ Optimizador             │ Clase Optimizer_SGD  │ optimizers.SGD()        │
│ Forward pass            │ layer.forward(X)     │ modelo.predict(X)       │
│ Cálculo de gradientes   │ Manual (backprop)    │ Automático              │
│ Actualización de pesos  │ optimizer.update()   │ Automático              │
│ Entrenamiento completo  │ Loop manual          │ modelo.fit()            │
└─────────────────────────┴──────────────────────┴─────────────────────────┘

LINEAS DE CODIGO:
  - Manual:     ~150-200 líneas (todas las clases)
  - TensorFlow: ~10 líneas (sin contar prints)

VENTAJAS DE TENSORFLOW:
  ✓ Código mucho más conciso
  ✓ Gradientes calculados automáticamente
  ✓ Optimizado para GPU/TPU
  ✓ Menos propenso a errores
  ✓ Fácil de escalar a redes más complejas

VENTAJAS DE ENTENDER LA IMPLEMENTACION MANUAL:
  ✓ Sabes qué hace cada componente
  ✓ Puedes diagnosticar problemas
  ✓ Entiendes las limitaciones
  ✓ Puedes crear componentes personalizados
  ✓ Tomas mejores decisiones de diseño
"""

print(comparacion)


RED NEURONAL CON TENSORFLOW/KERAS

Datos de entrada:
X shape: (3, 4) (3 muestras, 4 caracteristicas)
y shape: (3, 3) (3 muestras, 3 clases)

ARQUITECTURA DEL MODELO:
----------------------------------------------------------------------


C:\Users\esteb\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ capa_oculta (Dense)                  │ (None, 5)                   │              25 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ capa_salida (Dense)                  │ (None, 3)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 43 (172.00 B)

 Trainable params: 43 (172.00 B)

 Non-trainable params: 0 (0.00 B)

----------------------------------------------------------------------

MODELO COMPILADO:
  - Optimizador: SGD (learning_rate=0.5)
  - Funcion de perdida: Categorical Crossentropy
  - Metricas: Accuracy

PREDICCIONES ANTES DEL ENTRENAMIENTO (pesos aleatorios)

Predicciones (probabilidades):
  Muestra 0: [0.0793648 0.6858896 0.2347456] -> Clase 1 | Real: Clase 0 | ✗
  Muestra 1: [0.44278055 0.31846198 0.23875745] -> Clase 0 | Real: Clase 1 | ✗
  Muestra 2: [0.03681113 0.5655071  0.39768174] -> Clase 1 | Real: Clase 2 | ✗

Perdida inicial: 1.5334

ENTRENAMIENTO

Entrenamiento completado: 100 epocas
Perdida final: 0.0172
Accuracy final: 1.0000

Evolucion de la perdida:
  Epoca   1: Perdida = 1.5334
  Epoca  20: Perdida = 0.1165
  Epoca  40: Perdida = 0.0505
  Epoca  60: Perdida = 0.0310
  Epoca  80: Perdida = 0.0223
  Epoca 100: Perdida = 0.0172

PREDICCIONES DESPUES DEL ENTRENAMIENTO

Predicciones (probabilidades):
  Muestra 0: [9.9876606e-01 9.7932061e-04 2.5468459e-04] -> Clase 0 | Rea